In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import seaborn as sns

In [ ]:
df = pd.read_csv('FullDayWithAlarms.csv', header=1)
df

In [ ]:
df.describe().T

cod_gioco: Always 8 → probably identifies a single system/machine. to be dropped

intervallo_acquisizione: Always 60 seconds → suggests a fixed time granularity (1 minute). to be dropped

numero_transazioni: Average ≈ 2246, 204 to 5057 → reflects the intensity of activities in one minute.

tempo_min, tempo_max, tempo_medio: 
- tempo_min has very small values (average ≈ 2.7 sec).

- tempo_max has a huge maximum value (92682 sec ≈ 25h!) → obvious outlier. (not to be considered at the moment)

- tempo_medio follows a more regular pattern, but also has an outlier maximum (2019 sec).

numero_retry: almost always 0 but with a max of 6496 → may indicate errors or temporary technical problems.

numero_transazioni_errate: 81 to 448 → possible metric of service quality.


in addition to cod game and interval which we drop as constant, we will also drop time min and time max as among the 3 time measures we prefer to keep average time as there is a single unique observation for each minute of the original dataset.
time min and max in addition to being unclear and not very representative, we remove them after the data augmentation because they are useful for clipping the average time, once the data augmentation is over we drop time min and max so that we are left only with the 4 features we are interested in: number of transactions, average time, number of retries and number of wrong transactions.

In [ ]:
print(df.nunique())

for colonna in df.columns:
    valori_unici = df[colonna].unique()
    print(f"Valori univoci nella colonna '{colonna}':")
    print(valori_unici)
    print("---")

In [ ]:
# 1. Rename columns
df = df.rename(columns={
    'DATA ORA': 'data_ora',
    'INTERVALLO\nACQUISIZIONE': 'intervallo',
    'NUMERO\nTRANSAZIONI': 'numero_transazioni',
    'TEMPO MIN': 'tempo_min',
    'TEMPO MAX': 'tempo_max',
    'TEMPO MEDIO': 'tempo_medio',
    'NUMERO RETRY': 'numero_retry',
    'NUMERO \nTRANSAZIONI ERRATE': 'numero_transazioni_errate'
})

# 2. Remove costant columns
df = df.drop(columns=['COD \nGIOCO', 'intervallo'])

# 3. Conversion in datetime 
df['data_ora'] = pd.to_datetime(df['data_ora'], dayfirst=True)


int_cols = [
    'numero_transazioni', 'numero_retry', 'numero_transazioni_errate']
df[int_cols] = df[int_cols].astype(int)
df['tempo_medio'] = df['tempo_medio'].astype(float)

# . Controllo rapido
print(df.info())
print(df.head())

In [ ]:
df['data_ora'].min(), df['data_ora'].max()

In [ ]:
# -------------------------------------------------
# 1. Figure & axis with white background
# -------------------------------------------------
fig, ax = plt.subplots(figsize=(14, 6), facecolor='white')
ax.set_facecolor('white')               # axis background

# -------------------------------------------------
# 2. Plot the time-series
# -------------------------------------------------
ax.plot(df['data_ora'], df['numero_transazioni'], label='numero_transazioni', color='green')
ax.plot(df['data_ora'], df['numero_retry'], label='numero_retry', color='orange')
ax.plot(df['data_ora'], df['numero_transazioni_errate'], label='numero_transazioni_errate', color='red')
ax.plot(df['data_ora'], df['tempo_medio'], label='tempo_medio', color='blue')

# -------------------------------------------------
# 3. Design
# -------------------------------------------------
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate(rotation=45)
ax.set_xlabel('Date / Time')
ax.set_ylabel('Count / Time')
ax.set_title('Valid, retry and errored transactions over time')
ax.legend(loc='upper left')
ax.grid(True, linestyle='--', linewidth=0.5, color='grey', alpha=0.4)
plt.tight_layout()
plt.show()


In [8]:
df.to_csv("day_dataset.csv", index=False)

In [ ]:
# ========================================================
#  DATA-AUG 07:00 ➜ 02:59  (pausa 03:00-06:59)
# ========================================================

# --------------------------------------------------------
# 1) MINUTE-BY-MINUTE STATISTICS  (dtype → float)
# --------------------------------------------------------
def build_minute_stats(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['min_of_day'] = (
        (df['data_ora'] - df['data_ora'].dt.normalize())
        .dt.total_seconds() // 60
    ).astype(int)

    stats = (
        df.groupby('min_of_day')
          .agg(['min', 'max', 'mean', 'std'])
    )
    stats.columns = ['_'.join(col) for col in stats.columns]

    # --- extend 00:00-02:59 copying 22:00-23:59 ------------
    base_block = stats.loc[1380:1439]                     # 23:00-23:59
    night_idx  = range(0, 180)
    reps       = np.tile(base_block.to_numpy(), (3, 1))[:180]

    stats_night = pd.DataFrame(reps, index=night_idx, columns=stats.columns)
    stats_ext   = pd.concat([stats_night, stats.loc[420:]], axis=0).sort_index()

    stats_ext = stats_ext.apply(pd.to_numeric, errors='coerce')

    return stats_ext

# --------------------------------------------------------
# 2) SYNTHETIC DAY 07-03
# --------------------------------------------------------
def synth_day(stats: pd.DataFrame,
              day_shift: int,
              slow: bool = False,
              rng:  np.random.Generator = np.random.default_rng()):
    scale_rng      = (0.70, 0.85) if slow else (0.90, 1.05)
    noise_frac     = 0.005        if slow else 0.010
    anomaly_chance = 0.005        if slow else 0.020

    SPIKE_RATIO = 0.90
    SPIKE_RANGE = (1.10, 1.25)
    DROP_RANGE  = (0.85, 0.95)
    FLOOR_PCT   = 0.95

    cols_int   = ['numero_transazioni','tempo_min','tempo_max',
                  'numero_retry','numero_transazioni_errate']
    cols_float = ['tempo_medio']

    sampled = {}
    for c in cols_int + cols_float:
        mu    = stats[f'{c}_mean'].to_numpy(float)
        sigma = stats[f'{c}_std' ].fillna(0).to_numpy(float)
        sigma = np.where(sigma == 0, noise_frac * np.maximum(1, mu), sigma)
        base  = rng.normal(mu, sigma)
        scale = rng.uniform(*scale_rng, size=len(base))
        sampled[c] = base * scale

    out           = pd.DataFrame(sampled, index=stats.index)
    out[cols_int] = out[cols_int].round().astype(int).clip(lower=0)

    # clip time
    out['tempo_min']   = out['tempo_min' ].clip(lower=1)
    out['tempo_max']   = np.maximum(out['tempo_max'], out['tempo_min'] + 1)
    out['tempo_medio'] = out['tempo_medio'].clip(out['tempo_min'], out['tempo_max'])

    
    base_dt   = stats.attrs['base_dt'].normalize() + pd.Timedelta(days=day_shift)
    out['data_ora'] = base_dt + pd.to_timedelta(out.index, unit='m')
    early_mask = out.index < 420
    out.loc[early_mask, 'data_ora'] += pd.Timedelta(days=1)

    # --------------------- ANOMALIES ----------------------
    hour = (out.index // 60).to_numpy()

    busy_mask  = (11 <= hour) & (hour <= 14) | (17 <= hour) & (hour <= 21)
    night_mask = (hour < 3)                                    # 00-02
    
    scale_factor = np.where(busy_mask, 3.0,
                     np.where(night_mask, 0.5, 1.0))
    p_anom = np.clip(anomaly_chance * scale_factor, 0, 1)

    flag_anom = rng.random(len(out)) < p_anom
    is_spike  = rng.random(len(out)) < SPIKE_RATIO

    spike_f   = rng.uniform(*SPIKE_RANGE, len(out))
    drop_f    = rng.uniform(*DROP_RANGE , len(out))

    mask_spike = flag_anom &  is_spike
    mask_drop  = flag_anom & ~is_spike

    out.loc[mask_spike, 'numero_transazioni'] = (
        out.loc[mask_spike, 'numero_transazioni'] * spike_f[mask_spike]
    ).round().astype(int)

    floor = (stats['numero_transazioni_min'] * FLOOR_PCT).to_numpy()
    new_vals = (out.loc[mask_drop, 'numero_transazioni'] * drop_f[mask_drop]).round()
    out.loc[mask_drop, 'numero_transazioni'] = (
        np.maximum(new_vals, floor[mask_drop]).astype(int)
    )

    out['is_anomaly'] = flag_anom.astype(int)
    out = out[(out.index < 180) | (out.index >= 420)].reset_index(drop=True)
    return out

# --------------------------------------------------------
# 3) WEEK MAKER
# --------------------------------------------------------
def make_week(df_original: pd.DataFrame,
              rng: np.random.Generator = np.random.default_rng(42)) -> pd.DataFrame:
    stats = build_minute_stats(df_original)
    stats.attrs['base_dt'] = (
        df_original['data_ora'].dt.normalize().iloc[0] + pd.Timedelta(hours=7)
    )

    week = [df_original.assign(is_anomaly=0)]
    for d in range(1, 7):
        week.append(synth_day(stats, d, slow=(d == 3), rng=rng))

    return (pd.concat(week, ignore_index=True)
              .sort_values('data_ora')
              .reset_index(drop=True))


df_week  = make_week(df)

print(f"Dataset finale: {df_week.shape[0]:,} righe "
      f"(1 reale + 6 sintetiche) – dal {df_week['data_ora'].min()} "
      f"al {df_week['data_ora'].max()}")

# esempio rapido di verifica tipi
assert df_week['tempo_medio'].dtype == float
assert all(df_week[c].dtype == int for c in
           ['numero_transazioni','tempo_min','tempo_max',
            'numero_retry','numero_transazioni_errate'])


### Knobs to Intervene on for Modifying Data Augmentation

| Parameter | Location | Qualitative Effect | Typical Range |
|-----------|---------------|---------------------|--------------|
| **`scale_rng = (low, high)`** | `synth_day()` | *Multiplicative factor* common to **all** columns.<br>Shifts the entire distribution upwards ( > 1 ) or downwards ( < 1 ). | 0.6 – 1.4 |
| **`noise_frac`** | `synth_day()` | Amplitude of **additive** Gaussian noise (σ ≈ `noise_frac · real_std`).<br>Increases minute-by-minute "graininess". | 0 – 0.05 |
| **`anomaly_chance`** | `synth_day()` | % of records that become anomalies (spike/drop). | 0 – 0.10 |
| **`scale_factor`** (within `synth_day`, replacing `peak_boost`) | `synth_day()` inside the anomaly generation block | Multiplier for `anomaly_chance` during busy hours (e.g., 11-14 / 17-21) and potentially a reducer for off-peak hours. <br> *Original logic used `peak_boost = 1.3` as a fixed multiplier. The provided code uses a dynamic `scale_factor`.* | e.g., 0.5 (night), 1.0 (normal), 3.0 (busy) |
| **`SPIKE_RATIO`** | `synth_day()` | Probability that an anomaly is a spike (vs. a drop). | 0 – 1 |
| **`SPIKE_RANGE = (1.10, 1.25)`**<br>**`DROP_RANGE  = (0.85, 0.95)`** | `synth_day()` | Intensity of spikes or drops. | As desired |
| **Sampling Distribution** | Vectorized sampling block in `synth_day()` | Default is **Normal** N(μ, σ).<br>Can be replaced with Uniform, Gamma, etc. | — |

In [10]:
df_week = df_week.drop(columns=['tempo_min', 'tempo_max'])

In [11]:
df_week.to_csv("week_dataset.csv", index=False)

In [ ]:
print(df_week.info())
print(df_week.head())

In [ ]:
# 1. Split original and augmented days
df_original_day   = df_week[df_week['data_ora'].dt.date == df_week['data_ora'].dt.date.min()]
df_augmented_days = df_week[df_week['data_ora'].dt.date != df_week['data_ora'].dt.date.min()]
df_anomalies      = df_week[df_week['is_anomaly'] == 1]


fig, ax = plt.subplots(figsize=(15, 6))   

# 3. Plot original
ax.plot(
    df_original_day['data_ora'],
    df_original_day['numero_transazioni'],
    label="Original Day",
    linestyle='--',
    color='orange',
    alpha=0.9
)

# 4. Plot augmented
ax.plot(
    df_augmented_days['data_ora'],
    df_augmented_days['numero_transazioni'],
    label="Augmented Days",
    linestyle='--',
    color='steelblue',
    alpha=0.7
)

# 5. Highlight anomalies (red dots)
ax.scatter(
    df_anomalies['data_ora'],
    df_anomalies['numero_transazioni'],
    color='red',
    label='Injected Anomalies',
    zorder=5
)

ax.set_title("numero_transazioni: Original Day vs Augmented Days with Anomalies")
ax.set_xlabel("Ora")
ax.set_ylabel("Transactions")
ax.legend()
ax.grid(True)

# 7.  ⌚︎ Formatter solo HH:MM  (e locator ogni 2 h, opz.)
ax.xaxis.set_major_locator(mdates.HourLocator(interval=3))      # mostra 07:00, 10:00, …
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

plt.setp(ax.get_xticklabels(), rotation=60, ha='right')

fig.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

df_w = pd.read_csv('week_dataset.csv')
df_original_raw = pd.read_csv('day_dataset.csv')

# Copy for safety
df_processed = df_w.copy()

df_processed['data_ora'] = pd.to_datetime(df_processed['data_ora'])

print("\n--- General Information about DataFrame df_week ---")
df_processed.info()

print("\n--- Descriptive Statistics ---")
print(df_processed.describe().T)

print("\n--- Visualization of Numerical Feature Distributions ---")
numerical_cols_to_plot = ['numero_transazioni', 'tempo_medio', 'numero_retry', 'numero_transazioni_errate']

for col in numerical_cols_to_plot:
    plt.figure(figsize=(12, 5))
    
    # Histogram of the augmented dataset (df_processed)
    sns.histplot(df_processed[col], kde=True, bins=50, color="blue", label='Augmented Data (df_week)', stat="density")
    
    # Overlay histogram of the original dataset (df_original_raw) if available
    if df_original_raw is not None and col in df_original_raw.columns:
        sns.histplot(df_original_raw[col], kde=True, bins=50, color="lightyellow", label='Original Day Data', stat="density", alpha=0.7)
        # Alternative for a more "background" visualization for the original:
        # sns.kdeplot(df_original_raw[col], color="orange", label='Original Day KDE', fill=True, alpha=0.3, linewidth=0)


    plt.title(f'Distribution of {col}: Original vs. Augmented')
    plt.xlabel(col)
    plt.ylabel('Density') # Changed to Density to better compare different shapes
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()
    
    # Boxplots remain useful to see the dispersion and outliers of df_processed
    plt.figure(figsize=(12, 3))
    sns.boxplot(x=df_processed[col], color="blue", boxprops=dict(edgecolor='black', linewidth=1.5),  # Box properties
        whiskerprops=dict(color='green', linestyle='--', linewidth=1.5),  # Whisker properties
        medianprops=dict(color='red', linewidth=2),  # Median properties
        capprops=dict(color='purple', linewidth=1.5),  # Cap properties
        flierprops=dict(marker='o', markerfacecolor='orange', markersize=8, markeredgecolor='gray')  # Outlier properties
    )
    plt.title(f'Boxplot of {col} (Augmented Data)')
    plt.xlabel(col)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()
    
    plt.figure(figsize=(12, 3))
    sns.boxplot(x=df_original_raw[col], color="lightyellow", boxprops=dict(edgecolor='black', linewidth=1.5),  # Box properties
        whiskerprops=dict(color='green', linestyle='--', linewidth=1.5),  # Whisker properties
        medianprops=dict(color='red', linewidth=2),  # Median properties
        capprops=dict(color='purple', linewidth=1.5),  # Cap properties
        flierprops=dict(marker='o', markerfacecolor='orange', markersize=8, markeredgecolor='gray')  # Outlier properties
    )
    plt.title(f'Boxplot of {col}') # Kept original title as it refers to df_original_raw
    plt.xlabel(col)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()

print("\n--- Analysis of Propagated Outliers (Focus on tempo_max and tempo_medio) ---")
# The augmentation code might have limited extreme outliers,
# but let's check the maximum generated values.
print("Maximum values per column in week_dataset:")
print(df_processed[numerical_cols_to_plot].max())
print("\nMinimum values per column in week_dataset:")
print(df_processed[numerical_cols_to_plot].min())
# Compare these with the maximums/minimums of the original df if necessary.
# The `synth_day` function has clipping logic, which should have helped.
print("\nMaximum values per column in day_dataset:")
print(df_original_raw[numerical_cols_to_plot].max())
print("\nMinimum values per column in day_dataset:")
print(df_original_raw[numerical_cols_to_plot].min())


In [ ]:
print("\n--- Correlations between Selected Features ---")
selected_features_for_corr = ['numero_transazioni', 'tempo_medio', 'numero_retry', 'numero_transazioni_errate']
correlation_matrix = df_processed[selected_features_for_corr].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Selected Feature Correlation Matrix')
plt.show()

In [16]:
#run in the terminal -> mv week_dataset.csv data/

In [ ]:
# Cell: Library Imports

import os
import time
from datetime import datetime
from pathlib import Path
import json
from typing import Tuple, Dict, Any, List 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    LSTM, Dense, Input, Dropout, Conv1D, BatchNormalization, 
    Add, Activation, GlobalAveragePooling1D, RepeatVector, 
    TimeDistributed, Bidirectional
)
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, 
    LearningRateScheduler, TensorBoard
)

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    confusion_matrix, roc_curve, auc, 
    precision_recall_curve, average_precision_score,
    precision_score, recall_score, f1_score, 
    classification_report, roc_auc_score,
    mean_squared_error, mean_absolute_error, r2_score
)

import joblib

print("Libraries imported successfully.")

### Notebook Setup and Configuration

This notebook implements an anomaly detection system for time series data. 
The first step is to define all configuration parameters and ensure necessary directories for data, trained models, and results are created.

**What this cell does:**
*   Imports `os` and `pathlib.Path` for path manipulation.
*   Defines `PROJECT_ROOT` as the current working directory of the notebook.
*   Sets up paths for `DATA_DIR`, `MODELS_DIR`, and `RESULTS_DIR` relative to `PROJECT_ROOT`.
*   Creates these directories if they do not already exist, ensuring the project has a standardized structure for storing artifacts.
*   Defines crucial data parameters:
    *   `DATA_FILE`: The name of the augmented dataset file (`week_dataset.csv`).
    *   `TARGET_COLUMN`: The name of the column indicating anomalies (`is_anomaly`).
    *   `FEATURE_COLUMNS`: A list of feature names used for model training and analysis.
    *   `PREDICTION_FEATURES`: A subset of `FEATURE_COLUMNS` specifically targeted by forecasting models.
*   Specifies time series parameters:
    *   `SEQUENCE_LENGTH`: The number of past time steps (minutes) to use as input for sequence models (e.g., 60 minutes).
    *   `FORECAST_HORIZON`: The number of future time steps (minutes) to predict (e.g., 30 minutes).
*   Defines hyperparameters and settings for various models:
    *   `LSTM_PARAMS`: Parameters for the LSTM forecasting model, including units, dropout, batch size, epochs, etc. *Epochs are currently set to 1 for quick demonstration runs.*
    *   `TCN_PARAMS`: Parameters for the TCN forecasting model. *Epochs are currently set to 1.*
    *   `AUTOENCODER_PARAMS`: Parameters for the Autoencoder anomaly detection model, including layer units, dropout, and thresholding strategy. *Epochs are currently set to 1.*
*   Sets training data split sizes (`TRAIN_SIZE`, `VAL_SIZE`, `TEST_SIZE`) in terms of number of "days" from the augmented dataset.
*   Defines evaluation metrics to be used for forecasting and anomaly detection.
*   Configures visualization parameters (`PLOT_PARAMS`) for consistent styling of plots generated by `matplotlib` and `seaborn`.
*   Sets up print settings (`PRINT_PARAMS`) to control the verbosity of output during script execution (e.g., showing model summaries, timing).
*   Defines TensorBoard parameters (`TENSORBOARD_PARAMS`) for monitoring model training.

**Why these choices were made:**
*   Centralizing configuration makes the project easier to manage, modify, and experiment with.
*   Defining paths ensures that data, models, and results are stored in a consistent and predictable manner.
*   Specifying model hyperparameters allows for easy tuning and reproducibility of model training.
*   Setting epochs to 1 is for rapid execution during development or demonstration; for actual training, these would be higher (e.g., 50 as in the original scripts).
*   Standardized plotting and print parameters ensure consistent output across the notebook.

**Expected Output:**
This cell will print messages confirming that the configuration parameters have been loaded and that the necessary project directories (`data`, `models`, `results`) have been created or already exist. It will also print the absolute paths to these directories.

# Config

In [ ]:
# Cell 1: Configuration Parameters
import os
from pathlib import Path


PROJECT_ROOT = Path.cwd() 
DATA_DIR = PROJECT_ROOT / "data"
MODELS_DIR = PROJECT_ROOT / "models"
RESULTS_DIR = PROJECT_ROOT / "results"


for dir_path in [DATA_DIR, MODELS_DIR, RESULTS_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True) 

# Data parameters
DATA_FILE = "week_dataset.csv" 
TARGET_COLUMN = "is_anomaly"
FEATURE_COLUMNS = [
    "numero_transazioni",
    "tempo_medio",
    "numero_retry",
    "numero_transazioni_errate"
]

# Prediction features (subset of FEATURE_COLUMNS)
PREDICTION_FEATURES = [
    "numero_transazioni",
    "numero_transazioni_errate"
]

# Time series parameters
SEQUENCE_LENGTH = 60  # Number of time steps to look back (60 minutes)
FORECAST_HORIZON = 30  # Number of time steps to predict ahead (30 minutes)

# Model parameters
LSTM_PARAMS = {
    "units": 64,  
    "dropout": 0.3,  
    "recurrent_dropout": 0.2, 
    "batch_size": 64,  
    "epochs": 50,  
    "patience": 10,  
    "learning_rate": 0.001,
    "beta_1": 0.9,
    "beta_2": 0.999,
    "epsilon": 1e-07
}

TCN_PARAMS = {
    "nb_filters": 64,  
    "kernel_size": 4,
    "nb_stacks": 3,  
    "dilations": [1, 2, 4, 8],  
    "batch_size": 64,  
    "epochs": 50,  
    "patience": 10,  
    "learning_rate": 0.001,
    "beta_1": 0.9,
    "beta_2": 0.999,
    "epsilon": 1e-07
}

AUTOENCODER_PARAMS = {
    "units": [64, 32, 16], 
    "dropout": 0.2,
    "batch_size": 64,
    "epochs": 50,
    "patience": 3, 
    "learning_rate": 0.001,
    "beta_1": 0.9,
    "beta_2": 0.999,
    "epsilon": 1e-07,
    "reconstruction_threshold_percentile": 95,
    "forecast_horizon": 30, 
    "threshold_analysis_percentiles": [75, 80, 85, 90, 92, 95, 97, 98, 99, 99.5],
    "threshold_optimization_metric": "f1"  
}

# Training parameters
TRAIN_SIZE = 5 # Number of days for training
VAL_SIZE = 1   # Number of days for validation
TEST_SIZE = 1  # Number of days for testing

# Evaluation parameters
ANOMALY_THRESHOLD = 0.95  
METRICS = {
    "forecasting": ["mse", "mae", "rmse"],
    "anomaly_detection": ["precision", "recall", "f1", "auc_roc"]
}

# Visualization parameters
PLOT_PARAMS = {
    "figsize": (12, 8),
    "dpi": 100,
    "rc": {
        "figure.figsize": (12, 8), 
        "figure.dpi": 100,        
        "font.size": 12,
        "axes.titlesize": 14,
        "axes.labelsize": 12,
        "xtick.labelsize": 10,
        "ytick.labelsize": 10,
        "legend.fontsize": 10,
        "lines.linewidth": 2,
        "lines.markersize": 6,
        "grid.alpha": 0.3
    },
    "context": "notebook", 
    "palette": "deep"      
}

# Print settings
PRINT_PARAMS = {
    "show_summary": True,
    "show_progress": True,
    "show_metrics": True,
    "show_timing": True,
    "precision": 4 
}

# TensorBoard settings
TENSORBOARD_PARAMS = {
    "log_dir": str(RESULTS_DIR / "logs"),
    "histogram_freq": 1,
    "write_graph": True,
    "write_images": True,
    "update_freq": "epoch",
    "profile_batch": "500,520" 
} 

print("Configuration parameters loaded and directories ensured.")
print(f"Data directory: {DATA_DIR}")
print(f"Models directory: {MODELS_DIR}")
print(f"Results directory: {RESULTS_DIR}")

### Utility Functions

This cell defines a collection of helper functions that will be used throughout the notebook for various tasks such as data loading, preprocessing, sequence creation, model saving/loading, and plotting.

**What this cell does:**
*   **`load_and_preprocess_data()`**:
    *   Loads the dataset specified by `DATA_FILE` from the `DATA_DIR`.
    *   Converts the `data_ora` column to datetime objects.
    *   Sets the `data_ora` column as the DataFrame index.
    *   *Purpose:* To provide a standardized way to load and perform initial a_ora` column as the DataFrame index.
    *   *Purpose:* To provide a standardized way to load and perform initial common preprocessing steps on the raw dataset.
*   **`create_sequences(data, seq_length)`**:
    *   Takes a NumPy array of time series data and a sequence length.
    *   Generates input sequences (X) of `seq_length` and corresponding target values (y) for the next time step.
    *   *Purpose:* To transform time series data into a supervised learning format suitable for sequence models like LSTMs and TCNs.
*   **`split_data_chronological(df)`**:
    *   Splits the input DataFrame chronologically into training, validation, and test sets based on the `TRAIN_SIZE`, `VAL_SIZE`, and `TEST_SIZE` (number of days) defined in the configuration.
    *   It identifies unique days in the dataset and partitions the data accordingly.
    *   *Purpose:* To ensure a realistic evaluation scenario for time series models by preventing data leakage from future periods into the training or validation sets.
*   **`scale_data(train_data, val_data, test_data)`**:
    *   Initializes a `StandardScaler` from `sklearn.preprocessing`.
    *   Fits the scaler *only* on the `FEATURE_COLUMNS` of the training data.
    *   Transforms the training, validation, and test datasets using the fitted scaler.
    *   *Purpose:* To standardize the features to have zero mean and unit variance, which often improves the performance and stability of machine learning models, especially neural networks.
*   **Plotting Functions (`plot_forecast_results`, `plot_anomaly_scores`, `plot_confusion_matrix`, `plot_roc_curve`, `plot_precision_recall_curve`, `plot_anomaly_scores_distribution`)**:
    *   These functions provide standardized ways to visualize model performance for forecasting and anomaly detection tasks. They generate and save plots like actual vs. predicted, anomaly score distributions, confusion matrices, ROC curves, and Precision-Recall curves to the `RESULTS_DIR`.
    *   *Purpose:* To offer visual insights into model behavior and evaluation metrics.
*   **`calculate_metrics(y_true, y_pred)`**:
    *   Calculates common classification metrics (precision, recall, F1-score, AUC-ROC) given true and predicted binary labels. Handles potential `ValueError` for AUC if only one class is present.
    *   *Purpose:* To provide quantitative measures of anomaly detection performance. (Note: The original `utils.py` version was more geared towards classification; forecasting metrics are typically MSE, MAE, RMSE, calculated directly in their respective evaluation functions).
*   **`save_model(model, model_name)`**:
    *   Saves a trained model (Keras model or other types via `joblib`) to the `MODELS_DIR`.
    *   *Purpose:* To persist trained models for later use or deployment.
*   **`load_model(model_name)`**:
    *   Loads a previously saved model from the `MODELS_DIR`.
    *   *Purpose:* To retrieve trained models without needing to retrain them.
*   **`plot_anomaly_results(...)`**: A comprehensive plotting function specifically for anomaly detection, combining several visualizations. *This was present in the original `utils.py` and is slightly different from the individual plotting functions that were later moved into Cell 5. For a single notebook, these would be consolidated.*

**Why these functions were chosen/created:**
*   Encapsulating repetitive tasks into functions makes the main workflow cleaner, more readable, and less prone to errors.
*   Standardized data loading, splitting, and scaling ensure consistency across different modeling stages.
*   Reusable plotting functions allow for consistent visualization of results.

**Expected Output:**
This cell primarily defines functions. When executed, it will print "Utility functions defined." to confirm that the functions are now available in the notebook's namespace for subsequent cells to use. No other visual output is expected from this cell itself.

# Utils

In [ ]:
# Cell 2: Utility Functions

def load_and_preprocess_data() -> pd.DataFrame:
    """Load and preprocess the dataset."""
    
    df_path = DATA_DIR / DATA_FILE
    if not df_path.exists():
        raise FileNotFoundError(f"Data file not found: {df_path}. Please ensure '{DATA_FILE}' is in the '{DATA_DIR}' directory.")
    df = pd.read_csv(df_path)
    df['data_ora'] = pd.to_datetime(df['data_ora'])
    df.set_index('data_ora', inplace=True)
    return df

def create_sequences(data: np.ndarray, seq_length: int) -> Tuple[np.ndarray, np.ndarray]:
    """Create sequences for time series forecasting."""
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length)])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

def split_data_chronological(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Split data chronologically into train, validation, and test sets."""
    
    total_days_in_data = (df.index.max() - df.index.min()).days + 1 
    # This calculation of days_per_split might be problematic if data isn't perfectly contiguous or doesn't start at midnight
    # A more robust way if splits are by full days from the start of the dataset:
    unique_days = sorted(df.index.normalize().unique())
    
    if len(unique_days) < (TRAIN_SIZE + VAL_SIZE + TEST_SIZE):
        raise ValueError(f"Not enough unique days in the dataset ({len(unique_days)}) to satisfy the split sizes (Train:{TRAIN_SIZE}, Val:{VAL_SIZE}, Test:{TEST_SIZE}).")

    train_end_date = unique_days[TRAIN_SIZE - 1]
    val_end_date = unique_days[TRAIN_SIZE + VAL_SIZE - 1]

    train_data = df[df.index.normalize() <= train_end_date]
    val_data = df[(df.index.normalize() > train_end_date) & (df.index.normalize() <= val_end_date)]
    test_data = df[df.index.normalize() > val_end_date]
    
    
    print(f"Train data shape: {train_data.shape}, from {train_data.index.min()} to {train_data.index.max()}")
    print(f"Validation data shape: {val_data.shape}, from {val_data.index.min()} to {val_data.index.max()}")
    print(f"Test data shape: {test_data.shape}, from {test_data.index.min()} to {test_data.index.max()}")
    
    return train_data, val_data, test_data


def scale_data(train_data: pd.DataFrame, val_data: pd.DataFrame, test_data: pd.DataFrame) -> Tuple[StandardScaler, np.ndarray, np.ndarray, np.ndarray]:
    """Scale the data using StandardScaler."""
    
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train_data[FEATURE_COLUMNS])
    val_scaled = scaler.transform(val_data[FEATURE_COLUMNS])
    test_scaled = scaler.transform(test_data[FEATURE_COLUMNS])
    return scaler, train_scaled, val_scaled, test_scaled

def plot_forecast_results(actual: np.ndarray, predicted: np.ndarray, title: str, feature_names: List[str]):
    """Plot actual vs predicted values for each feature."""
    
    n_features = len(feature_names)
    if n_features == 0:
        print("No features to plot for forecast results.")
        return
    fig, axes = plt.subplots(n_features, 1, figsize=(12, 4*n_features), squeeze=False) # Ensure axes is always 2D
    
    for i, (ax, feature) in enumerate(zip(axes.ravel(), feature_names)):
        ax.plot(actual[:, i], label='Actual', alpha=0.7)
        ax.plot(predicted[:, i], label='Predicted', alpha=0.7)
        ax.set_title(f'{feature} - {title}')
        ax.legend()
        ax.grid(True)
    
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / f'forecast_{title.lower().replace(" ", "_")}.png')
    plt.close(fig) 

def plot_anomaly_scores(scores: np.ndarray, threshold: float, title: str):
    """Plot anomaly scores with threshold."""
    
    plt.figure(figsize=(12, 6))
    plt.plot(scores, label='Anomaly Score')
    plt.axhline(y=threshold, color='r', linestyle='--', label=f'Threshold: {threshold:.4f}')
    plt.title(f'Anomaly Scores - {title}')
    plt.legend()
    plt.grid(True)
    plt.savefig(RESULTS_DIR / f'anomaly_scores_{title.lower().replace(" ", "_")}.png')
    plt.close()

def calculate_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
    """Calculate evaluation metrics."""
    from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
    
    # Ensure y_true and y_pred are binary for these metrics
    y_true_binary = (y_true > 0.5).astype(int)
    y_pred_binary = (y_pred > 0.5).astype(int)

    # Handling cases where a class might be absent in predictions (e.g., no anomalies predicted)
    # or in true labels (though less likely for anomaly detection test sets with anomalies)
    # zero_division=0 will return 0.0 if a class is not present in predictions for precision/recall/f1
    # For roc_auc_score, it requires at least one sample from each class in y_true for a meaningful score.
    # If only one class present in y_true, roc_auc_score is undefined.

    precision = precision_score(y_true_binary, y_pred_binary, zero_division=0)
    recall = recall_score(y_true_binary, y_pred_binary, zero_division=0)
    f1 = f1_score(y_true_binary, y_pred_binary, zero_division=0)
    
    auc_roc_val = np.nan # Default to NaN if AUC cannot be computed
    if len(np.unique(y_true_binary)) > 1: # Check if more than one class in true labels
        try:
            auc_roc_val = roc_auc_score(y_true_binary, y_pred) # y_pred should be scores/probabilities for AUC
        except ValueError as e:
            print(f"Warning: Could not calculate AUC ROC: {e}. This often happens if y_pred contains only one class.")
            # Or if y_pred contains class labels instead of probabilities.
            # For anomaly detection, y_pred is often binary labels from a threshold, 
            # y_score (the raw anomaly scores) should be used for roc_auc_score.
            # This function might need y_score as an argument if used for anomaly detection AUC.
            # For now, assuming y_pred can be scores for some contexts.

    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc_roc': auc_roc_val
    }

def save_model(model: Any, model_name: str):
    """Save model to disk."""
    # Assuming MODELS_DIR is defined
    if isinstance(model, tf.keras.Model):
        model.save(MODELS_DIR / f'{model_name}.keras')
    else:
        import joblib
        joblib.dump(model, MODELS_DIR / f'{model_name}.joblib')
    print(f"Model '{model_name}' saved to {MODELS_DIR}")


def load_model(model_name: str) -> Any:
    """Load model from disk."""
    # Assuming MODELS_DIR is defined
    model_path_keras = MODELS_DIR / f'{model_name}.keras'
    model_path_joblib = MODELS_DIR / f'{model_name}.joblib'

    if model_path_keras.exists():
        print(f"Loading Keras model: {model_path_keras}")
        return tf.keras.models.load_model(model_path_keras)
    elif model_path_joblib.exists():
        import joblib
        print(f"Loading Joblib model: {model_path_joblib}")
        return joblib.load(model_path_joblib)
    else:
        raise FileNotFoundError(f"Model file not found for '{model_name}' in {MODELS_DIR}")


def plot_anomaly_results(y_true: np.ndarray, 
                        y_pred: np.ndarray, 
                        anomaly_scores: np.ndarray,
                        threshold: float,
                        model_name: str,
                        save_path: Path) -> None:
    """Plot anomaly detection results.
    
    Args:
        y_true: True labels (0 for normal, 1 for anomaly)
        y_pred: Predicted labels (binary)
        anomaly_scores: Anomaly scores for each sample
        threshold: Anomaly threshold used to derive y_pred
        model_name: Name of the model
        save_path: Path to save the plots
    """
    # Ensure y_true is binary
    y_true_binary = (np.any(y_true > 0, axis=1)).astype(int) if len(y_true.shape) > 1 and y_true.shape[1] > 1 else (y_true > 0.5).astype(int)
    save_path.mkdir(parents=True, exist_ok=True)

    # Plot 1: Anomaly Score Distribution
    plt.figure(figsize=(10, 6))
    df_scores = pd.DataFrame({
        'Score': anomaly_scores,
        'Class': ['Anomaly' if y == 1 else 'Normal' for y in y_true_binary]
    })
    sns.histplot(data=df_scores, x='Score', hue='Class', kde=True, 
                 palette={'Normal': 'blue', 'Anomaly': 'red'}, alpha=0.6)
    plt.axvline(x=threshold, color='r', linestyle='--', label=f'Threshold: {threshold:.4f}')
    plt.title(f'{model_name} - Anomaly Scores Distribution')
    plt.xlabel('Anomaly Score')
    plt.ylabel('Count')
    plt.legend()
    plt.savefig(save_path / f'{model_name}_Anomaly_Scores_Distribution.png') # Adjusted filename
    plt.close()
    
    # Plot 2: Confusion Matrix
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_true_binary, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{model_name} - Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path / f'{model_name}_Confusion_Matrix.png') # Adjusted filename
    plt.close()
    
    # Plot 3: ROC Curve
    if len(np.unique(y_true_binary)) > 1: # ROC AUC is meaningful only with multiple classes
        plt.figure(figsize=(8, 6))
        fpr, tpr, _ = roc_curve(y_true_binary, anomaly_scores) # Use scores for ROC
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.3f})')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'{model_name} - ROC Curve')
        plt.legend(loc="lower right")
        plt.savefig(save_path / f'{model_name}_ROC_Curve.png') # Adjusted filename
        plt.close()
    else:
        print(f"Skipping ROC curve for {model_name} as y_true_binary contains only one class.")

    # Plot 4: Precision-Recall Curve
    plt.figure(figsize=(8, 6))
    precision_curve, recall_curve, _ = precision_recall_curve(y_true_binary, anomaly_scores) # Use scores for PR curve
    avg_precision = average_precision_score(y_true_binary, anomaly_scores)
    plt.plot(recall_curve, precision_curve, label=f'PR curve (AP = {avg_precision:.3f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'{model_name} - Precision-Recall Curve')
    plt.legend(loc="lower left")
    plt.savefig(save_path / f'{model_name}_Precision_Recall_Curve.png') # Adjusted filename
    plt.close()
    
    # Plot 5: Threshold Analysis (if CSV exists)
    threshold_csv_path = RESULTS_DIR / 'autoencoder_threshold_analysis.csv' # Assuming it's in RESULTS_DIR
    if threshold_csv_path.exists(): # Check if RESULTS_DIR is defined
        threshold_df = pd.read_csv(threshold_csv_path)
        plt.figure(figsize=(10, 6))
        plt.plot(threshold_df['Threshold'], threshold_df['Precision'], label='Precision')
        plt.plot(threshold_df['Threshold'], threshold_df['Recall'], label='Recall')
        plt.plot(threshold_df['Threshold'], threshold_df['F1'], label='F1 Score')
        # Highlight the operational threshold used for y_pred
        plt.axvline(x=threshold, color='r', linestyle='--', label=f'Chosen Threshold: {threshold:.4f}')
        plt.xlabel('Threshold')
        plt.ylabel('Score')
        plt.title(f'{model_name} - Threshold Analysis (from Validation)')
        plt.legend()
        plt.grid(True)
        plt.savefig(save_path / f'{model_name}_Threshold_Analysis.png') # Adjusted filename
        plt.close()
    else:
        print(f"Threshold analysis CSV not found at {threshold_csv_path}, skipping plot.")

print("Utility functions defined.")

### Data Preparation Pipeline

This cell executes the main data preparation pipeline. It leverages the utility functions defined previously to load the augmented dataset, perform chronological splitting, scale the features, and create sequences suitable for both time series forecasting and anomaly detection models.

**What this cell does:**
1.  **Load and Preprocess:** Calls `load_and_preprocess_data()` to load `week_dataset.csv` and set the datetime index.
2.  **Chronological Split:** Calls `split_data_chronological()` to divide the dataset into training (5 days), validation (1 day), and test (1 day) sets.
3.  **Save Test Timestamps:** Saves the timestamps corresponding to the test set sequences. This is useful for aligning predictions with actual times during evaluation and plotting.
4.  **Forecasting Data Preparation:**
    *   Filters the training and validation sets to include *only normal data* (where `is_anomaly == 0`) for training the forecasting models. This ensures the models learn to predict normal behavior.
    *   Initializes and fits a `StandardScaler` (`scaler_forecast`) *only* on the features of the normal training data.
    *   Transforms the normal training, normal validation, and the full (mixed normal/anomaly) test data using this scaler.
    *   Saves `scaler_forecast.joblib`.
    *   Calls `create_sequences()` to generate input sequences (X) and target values (y) for LSTM/TCN models from the scaled forecasting data.
    *   Saves the `X_train_forecast`, `y_train_forecast`, `X_val_forecast`, `y_val_forecast`, `X_test_forecast`, and `y_test_forecast` arrays as `.npy` files in the `DATA_DIR`.
5.  **Anomaly Detection Data Preparation (for Autoencoder):**
    *   Filters the training set to include *only normal data* for training the autoencoder.
    *   Initializes and fits a separate `StandardScaler` (`scaler_ad`) *only* on the features of this normal training data.
    *   Transforms the normal training data, and the full (mixed normal/anomaly) validation and test data using this `scaler_ad`.
    *   Saves `scaler_ad.joblib`.
    *   Calls `create_sequences()` to generate input sequences for the autoencoder:
        *   `X_train_ad`: Sequences from scaled normal training data (target is to reconstruct these).
        *   `X_val_ad_mixed`, `X_test_ad_mixed`: Sequences from scaled mixed validation and test data.
        *   `y_val_ad_labels`, `y_test_ad_labels`: The corresponding true anomaly labels for the validation and test sequences.
    *   Saves these arrays as `.npy` files.
6.  **Save Split DataFrames:** Saves the raw (unscaled, unsequenced) `train_data`, `val_data`, and `test_data` DataFrames to CSV files for reference or other analyses.
7.  **Create Basic Visualizations:** Calls `create_visualizations()` to generate and save plots showing feature distributions, time series splits, correlation matrix of training data, and anomaly distribution across splits. These plots are saved in `results/visualizations_prepare_data/`.

**Why these steps are performed:**
*   **Separate Scalers:** Using distinct scalers for forecasting and anomaly detection, both fitted only on normal training data, is crucial. Forecasting models learn normal patterns, and the autoencoder must learn to reconstruct normal data with low error.
*   **Sequence Creation:** Time series models like LSTM, TCN, and sequence-based Autoencoders require input data to be structured as sequences of past observations.
*   **Saving Intermediate Data:** Saving processed data arrays and scalers allows for resuming experiments or using these preprocessed components in other parts of the project without rerunning the entire preparation pipeline.
*   **Initial Visualizations:** Help in understanding the data characteristics after splitting and identifying any potential issues before model training.

**Expected Output:**
This cell will print several messages indicating the progress of data loading, splitting, scaling, and sequence creation. It will also print:
*   Shapes of the train, validation, and test DataFrames.
*   Confirmation messages when scalers and data arrays are saved.
*   A final message "Data preparation completed successfully!".
*   Several plot files will be saved to the `results/visualizations_prepare_data/` directory (e.g., `split_feature_distributions.png`, `split_feature_timeseries.png`, etc.).
The function will return a dictionary `prepared_data_dict` containing all the processed DataFrames and NumPy arrays, which can be used by subsequent cells if needed.

# Phase0 data preparation

In [ ]:
# Cell 3: Data Preparation Script Logic
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import joblib

# Assuming config variables are defined in Cell 1 (config.py content)
# Assuming utility functions are defined in Cell 2 (utils.py content)

def prepare_data():
    """Main function to prepare the dataset for the project."""
    print("Loading and preprocessing data...")
    df = load_and_preprocess_data() # from utils
    
    # Create time-based features
    # df['hour'] = df.index.hour # This was in your original script, but if FEATURE_COLUMNS does not include them,
    # df['day_of_week'] = df.index.dayofweek # they won't be scaled or used by models unless explicitly added to FEATURE_COLUMNS.
                                          # For now, assuming FEATURE_COLUMNS are the primary ones.

    # Split data chronologically
    print("Splitting data into train, validation, and test sets...")
    train_data, val_data, test_data = split_data_chronological(df) # from utils
    
    # Save timestamps for later use (original full dataframe index)
    # This might be better if timestamps are saved for train, val, test separately or handled during evaluation.
    # For simplicity in a notebook, ensure `df.index` corresponds to the full dataset before splitting if that's the intent.
    # Or, save timestamps from test_data for final evaluation plotting.
    # Let's save the index of the test_data's sequences for precise plotting later.
    # Timestamps for X_test sequences will be (len(test_data) - SEQUENCE_LENGTH)
    test_timestamps_for_sequences = test_data.index[SEQUENCE_LENGTH:]
    np.save(DATA_DIR / 'test_timestamps_for_sequences.npy', test_timestamps_for_sequences.values)
    
    # Prepare data for forecasting (using only normal data for training)
    print("Preparing data for forecasting...")
    train_data_forecast_normal_only = train_data[train_data[TARGET_COLUMN] == 0].copy()
    val_data_forecast_normal_only = val_data[val_data[TARGET_COLUMN] == 0].copy() # Validation also on normal for forecasting model dev
    
    # Scale the features for forecasting
    scaler_forecast = StandardScaler()
    # Fit scaler ONLY on normal training data features
    train_scaled_forecast = scaler_forecast.fit_transform(train_data_forecast_normal_only[FEATURE_COLUMNS])
    # Transform validation and test data (test data includes anomalies, which is fine for evaluation AFTER model is trained on normal)
    val_scaled_forecast = scaler_forecast.transform(val_data_forecast_normal_only[FEATURE_COLUMNS]) # Scale normal validation data
    test_scaled_forecast = scaler_forecast.transform(test_data[FEATURE_COLUMNS]) # Scale all test data
    
    # Save the scaler for forecasting
    joblib.dump(scaler_forecast, MODELS_DIR / 'scaler_forecast.joblib')
    
    # Create sequences for time series forecasting
    print("Creating sequences for time series forecasting...")
    X_train_forecast, y_train_forecast = create_sequences(train_scaled_forecast, SEQUENCE_LENGTH)
    X_val_forecast, y_val_forecast = create_sequences(val_scaled_forecast, SEQUENCE_LENGTH)
    X_test_forecast, y_test_forecast = create_sequences(test_scaled_forecast, SEQUENCE_LENGTH)
    
    # Save forecasting data
    np.save(DATA_DIR / 'X_train_forecast.npy', X_train_forecast)
    np.save(DATA_DIR / 'y_train_forecast.npy', y_train_forecast)
    np.save(DATA_DIR / 'X_val_forecast.npy', X_val_forecast)
    np.save(DATA_DIR / 'y_val_forecast.npy', y_val_forecast)
    np.save(DATA_DIR / 'X_test_forecast.npy', X_test_forecast)
    np.save(DATA_DIR / 'y_test_forecast.npy', y_test_forecast)
    
    # Prepare data for anomaly detection (Autoencoder: train on normal, validate/test on mixed)
    print("Preparing data for anomaly detection (Autoencoder)...")
    train_data_ad_normal_only = train_data[train_data[TARGET_COLUMN] == 0].copy()
    
    # Scale the features for anomaly detection (Autoencoder)
    scaler_ad = StandardScaler()
    # Fit scaler ONLY on normal training data features
    train_scaled_ad_normal_only = scaler_ad.fit_transform(train_data_ad_normal_only[FEATURE_COLUMNS])
    # Transform validation and test data (these will contain anomalies)
    val_scaled_ad_mixed = scaler_ad.transform(val_data[FEATURE_COLUMNS])
    test_scaled_ad_mixed = scaler_ad.transform(test_data[FEATURE_COLUMNS])
    
    # Save the scaler for anomaly detection
    joblib.dump(scaler_ad, MODELS_DIR / 'scaler_ad.joblib')
    
    # Create sequences for anomaly detection
    # Autoencoder trains to reconstruct normal data
    X_train_ad_normal_only, _ = create_sequences(train_scaled_ad_normal_only, SEQUENCE_LENGTH) # y_train_ad is X_train_ad for autoencoders
    
    # For validation and testing, we need X and the original y (anomaly labels)
    X_val_ad_mixed, _ = create_sequences(val_scaled_ad_mixed, SEQUENCE_LENGTH) # We'll use the true anomalies from val_data
    y_val_ad_labels = val_data[TARGET_COLUMN].values[SEQUENCE_LENGTH:] # Align labels with sequences

    X_test_ad_mixed, _ = create_sequences(test_scaled_ad_mixed, SEQUENCE_LENGTH)
    y_test_ad_labels = test_data[TARGET_COLUMN].values[SEQUENCE_LENGTH:] # Align labels with sequences
    
    # Save anomaly detection data
    np.save(DATA_DIR / 'X_train_ad.npy', X_train_ad_normal_only) # Train autoencoder only on normal sequences
    np.save(DATA_DIR / 'X_val_ad.npy', X_val_ad_mixed)
    np.save(DATA_DIR / 'y_val_ad_labels.npy', y_val_ad_labels) # Save actual anomaly labels for validation
    np.save(DATA_DIR / 'X_test_ad.npy', X_test_ad_mixed)
    np.save(DATA_DIR / 'y_test_ad_labels.npy', y_test_ad_labels) # Save actual anomaly labels for test
    
    # Save original dataframes (already split) for reference
    train_data.to_csv(DATA_DIR / 'train_data.csv')
    val_data.to_csv(DATA_DIR / 'val_data.csv')
    test_data.to_csv(DATA_DIR / 'test_data.csv')
    
    
    print("Creating basic visualizations...")
    create_visualizations(train_data, val_data, test_data) # from utils
    
    print("Data preparation completed successfully!")
    
    return { 
        'train_data': train_data,
        'val_data': val_data,
        'test_data': test_data,
        'X_train_forecast': X_train_forecast,
        'y_train_forecast': y_train_forecast,
        'X_val_forecast': X_val_forecast,
        'y_val_forecast': y_val_forecast,
        'X_test_forecast': X_test_forecast,
        'y_test_forecast': y_test_forecast,
        'X_train_ad': X_train_ad_normal_only,
        'X_val_ad': X_val_ad_mixed,
        'y_val_ad_labels': y_val_ad_labels,
        'X_test_ad': X_test_ad_mixed,
        'y_test_ad_labels': y_test_ad_labels
    }

def create_visualizations(train_data, val_data, test_data):
    """Create basic visualizations of the data."""
    # Create directory for visualizations if it doesn't exist
    # Assuming RESULTS_DIR and FEATURE_COLUMNS, TARGET_COLUMN are defined
    vis_dir = RESULTS_DIR / 'visualizations_prepare_data' # Differentiate from other potential visualizations
    vis_dir.mkdir(parents=True, exist_ok=True)
    
    # Plot feature distributions
    plt.figure(figsize=(15, 10))
    for i, feature in enumerate(FEATURE_COLUMNS, 1):
        plt.subplot(2, 2, i)
        sns.histplot(train_data[feature], label='Train', alpha=0.5, kde=True)
        sns.histplot(val_data[feature], label='Validation', alpha=0.5, kde=True)
        sns.histplot(test_data[feature], label='Test', alpha=0.5, kde=True)
        plt.title(f'{feature} Distribution')
        plt.legend()
    plt.tight_layout()
    plt.savefig(vis_dir / 'split_feature_distributions.png') # Renamed from your list
    plt.close()
    
    # Plot time series of features
    plt.figure(figsize=(15, 10))
    for i, feature in enumerate(FEATURE_COLUMNS, 1):
        plt.subplot(2, 2, i)
        plt.plot(train_data.index, train_data[feature], label='Train', alpha=0.7)
        plt.plot(val_data.index, val_data[feature], label='Validation', alpha=0.7)
        plt.plot(test_data.index, test_data[feature], label='Test', alpha=0.7)
        plt.title(f'{feature} Time Series')
        plt.legend()
        plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(vis_dir / 'split_feature_timeseries.png') # Renamed from your list
    plt.close()
    
    # Plot correlation matrix (using training data)
    plt.figure(figsize=(10, 8))
    correlation_matrix = train_data[FEATURE_COLUMNS].corr()
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
    plt.title('Feature Correlation Matrix (Training Data)')
    plt.tight_layout()
    plt.savefig(vis_dir / 'feature_correlation_matrix_train_data.png') # Specific to train data
    plt.close()
    
    # Plot anomaly distribution
    plt.figure(figsize=(10, 6))
    anomaly_counts = pd.DataFrame({
        'Train': train_data[TARGET_COLUMN].value_counts(normalize=True) * 100, # Percentage
        'Validation': val_data[TARGET_COLUMN].value_counts(normalize=True) * 100,
        'Test': test_data[TARGET_COLUMN].value_counts(normalize=True) * 100
    }).fillna(0)
    
    anomaly_counts.plot(kind='bar')
    plt.title('Anomaly Distribution Across Splits (% of total in split)')
    plt.xlabel('Is Anomaly (0: Normal, 1: Anomaly)')
    plt.ylabel('Percentage of samples in split (%)')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.savefig(vis_dir / 'anomaly_distribution_splits_percentage.png') # Renamed from your list
    plt.close()

# Execute data preparation when this cell is run
prepared_data_dict = prepare_data()
print("\nData preparation script executed. Prepared data is available in `data`.")

### Phase 1: Time Series Forecasting Models (LSTM & TCN)

This section focuses on training and evaluating two types of neural networks for time series forecasting: Long Short-Term Memory (LSTM) and Temporal Convolutional Network (TCN). The goal is to predict future values of `numero_transazioni` (number of transactions) and `numero_transazioni_errate` (number of errored transactions) based on past observations. These models are trained exclusively on data identified as "normal" to learn baseline operational patterns.

**What this cell does:**
1.  **Setup:**
    *   Sets Matplotlib and Seaborn plotting styles using `PLOT_PARAMS` from the configuration cell.
    *   Configures TensorFlow global policies (disables JIT, sets mixed precision to 'float32' as per original script intent for this phase).
2.  **Define Helper Functions:**
    *   `print_training_info()`: Prints model summary and training start/elapsed time.
    *   `create_lstm_model()`: Defines the architecture for the LSTM model. It includes two LSTM layers (the first bidirectional if using `tf.keras.layers.Bidirectional(LSTM(...))`, though the provided code uses `LSTM(...)` directly), Batch Normalization, Dropout layers, and Dense layers for output. It uses Adam optimizer and Huber loss.
    *   `create_tcn_model()`: Defines the architecture for the TCN model. It uses a series of residual blocks with dilated causal convolutions, Batch Normalization, and Dropout. It also uses Adam optimizer and Huber loss.
    *   `prepare_data_for_training()`: Selects the target columns (`PREDICTION_FEATURES`) from the y-arrays for training the forecasting models.
    *   `train_forecasting_model()`:
        *   Takes a model, training/validation data, model name, and model-specific parameters.
        *   Implements callbacks: `EarlyStopping` (to prevent overfitting), `ModelCheckpoint` (to save the best model), `ReduceLROnPlateau` (to adjust learning rate), and `TensorBoard` (for logging).
        *   Uses `tf.data.Dataset` for efficient data input pipelines.
        *   Fits the model and returns the trained model, history, and training time.
    *   `evaluate_forecasting_model()`:
        *   Makes predictions on the test set.
        *   Calculates MSE, MAE, and RMSE metrics on scaled predictions.
        *   If a scaler is provided, it denormalizes the actual and predicted values for the `PREDICTION_FEATURES` and plots them.
        *   Plots the distribution of prediction errors (denormalized).
        *   Generates and plots future predictions for the next 30 minutes using a recursive approach.
        *   Saves predictions, future forecasts, and metrics to files.
3.  **`run_forecasting()` Function:**
    *   Loads the preprocessed and sequenced forecasting data (`X_train_forecast`, `y_train_forecast`, etc.) and the `scaler_forecast`.
    *   Prepares `y_train`, `y_val`, `y_test` to contain only the `PREDICTION_FEATURES`.
    *   Instantiates, trains, and evaluates both the LSTM and TCN models using the helper functions.
    *   Saves the trained LSTM and TCN models.
    *   Plots a comparison of their training histories (loss vs. epoch).
    *   Plots a bar chart comparing their evaluation metrics (MSE, MAE, RMSE) on the test set.
4.  **Execution:** Calls `run_forecasting()` to perform all the above steps.

**Why these choices were made:**
*   **LSTM and TCN:** Chosen for their proven effectiveness in modeling sequential data and capturing temporal dependencies.
*   **Training on Normal Data:** Forecasting models are trained on normal data to learn the system's baseline behavior. Significant deviations from these learned patterns during inference can indicate anomalies.
*   **Huber Loss:** More robust to outliers in time series data compared to MSE.
*   **Callbacks:** Essential for efficient and effective neural network training (preventing overfitting, saving best models, dynamic learning rate adjustment).
*   **Detailed Evaluation:** Includes plotting actual vs. predicted values, error distributions, and future forecasts to provide a comprehensive understanding of model performance.
*   **Model Comparison:** Directly comparing LSTM and TCN on the same task and data helps in selecting the better performing architecture for this specific problem.

**Expected Output:**
*   Print statements indicating the loading of data and scalers.
*   For both LSTM and TCN models:
    *   Model summaries (if `PRINT_PARAMS['show_summary']` is True).
    *   Training progress (epochs, loss, validation loss, etc., if `PRINT_PARAMS['show_progress']` is True).
    *   Training start/end times and elapsed times (if `PRINT_PARAMS['show_timing']` is True).
    *   Evaluation metrics (MSE, MAE, RMSE) on the test set (if `PRINT_PARAMS['show_metrics']` is True).
*   Saved plot files in the `results/` directory:
    *   `lstm_forecast_vs_actual.png` & `tcn_forecast_vs_actual.png`: Actual vs. Predicted values for target features.
    *   `lstm_error_distribution.png` & `tcn_error_distribution.png`: Distribution of prediction errors.
    *   `lstm_future_forecast.png` & `tcn_future_forecast.png`: Forecasts for the next 30 minutes.
    *   `forecasting_training_history.png`: Comparison of LSTM and TCN training/validation loss.
    *   `forecasting_model_comparison.png`: Bar chart comparing LSTM and TCN evaluation metrics.
*   Saved model files (`lstm_forecasting_model.keras`, `tcn_forecasting_model.keras`) in the `models/` directory.
*   Saved prediction arrays and metric JSON files in the `results/` directory.
*   A final message "Forecasting models trained and evaluated."
*   The cell will return a dictionary `forecasting_run_results` containing metrics and model objects.

# Phase1 forecasting

In [ ]:
# Cell 4: Time Series Forecasting Model Training and Evaluation

plt.rcParams.update(PLOT_PARAMS['rc'])
sns.set_context(PLOT_PARAMS['context'])
sns.set_palette(PLOT_PARAMS['palette'])

tf.config.optimizer.set_jit(False) 
tf.keras.mixed_precision.set_global_policy('float32') 


def print_training_info(model_name: str, model: Model, start_time: float):
    """Print training information."""
    if PRINT_PARAMS['show_summary']:
        print(f"\n{'='*50}")
        print(f"Training {model_name}")
        print(f"{'='*50}")
        model.summary(line_length=120) 
    
    if PRINT_PARAMS['show_timing']:
        elapsed_time = time.time() - start_time
        print(f"\nTraining started at: {datetime.fromtimestamp(start_time).strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Elapsed time: {elapsed_time:.2f} seconds")

def create_lstm_model(input_shape: Tuple[int, int]) -> Model:
    """Create LSTM model for time series forecasting."""
    model = Sequential([
        LSTM(LSTM_PARAMS['units'], 
             input_shape=input_shape,
             return_sequences=True,
             dropout=LSTM_PARAMS['dropout'],
             recurrent_dropout=0), 
        BatchNormalization(),
        LSTM(LSTM_PARAMS['units'] // 2,
             return_sequences=False,
             dropout=LSTM_PARAMS['dropout'],
             recurrent_dropout=0),
        BatchNormalization(),
        Dense(LSTM_PARAMS['units'] // 2, activation='relu'),
        Dropout(LSTM_PARAMS['dropout']),
        Dense(LSTM_PARAMS['units'] // 4, activation='relu'),
        Dropout(LSTM_PARAMS['dropout']),
        Dense(len(PREDICTION_FEATURES)) 
    ])
    
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=LSTM_PARAMS.get('learning_rate', 0.001), 
        beta_1=LSTM_PARAMS.get('beta_1', 0.9),
        beta_2=LSTM_PARAMS.get('beta_2', 0.999),
        epsilon=LSTM_PARAMS.get('epsilon', 1e-07)
    )
    model.compile(optimizer=optimizer, loss='huber')
    return model

def create_tcn_model(input_shape: Tuple[int, int]) -> Model:
    """Create Temporal Convolutional Network model."""
    def residual_block(x, dilation_rate, nb_filters, kernel_size, dropout_rate=0.1):
        y = Conv1D(filters=nb_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, padding='causal')(x)
        y = BatchNormalization()(y)
        y = Activation('relu')(y)
        y = Dropout(dropout_rate)(y)
        
        y = Conv1D(filters=nb_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, padding='causal')(y)
        y = BatchNormalization()(y)
        y = Dropout(dropout_rate)(y)
        
        if x.shape[-1] != nb_filters:
            x_reshaped = Conv1D(nb_filters, 1, padding='same')(x) 
        else:
            x_reshaped = x
        
        out = Add()([x_reshaped, y])
        out = Activation('relu')(out)
        return out
    
    inputs = Input(shape=input_shape)
    x = inputs
    
    for _ in range(TCN_PARAMS.get('nb_stacks', 1)): 
        for dilation_rate in TCN_PARAMS['dilations']:
            x = residual_block(
                x,
                dilation_rate=dilation_rate,
                nb_filters=TCN_PARAMS['nb_filters'],
                kernel_size=TCN_PARAMS['kernel_size'],
                dropout_rate=0.1 
            )
    
    x = GlobalAveragePooling1D()(x)
    
    x = Dense(TCN_PARAMS['nb_filters'] // 2, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    x = Dense(TCN_PARAMS['nb_filters'] // 4, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    outputs = Dense(len(PREDICTION_FEATURES))(x) 
    
    model = Model(inputs=inputs, outputs=outputs)
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=TCN_PARAMS.get('learning_rate', 0.001),
        beta_1=TCN_PARAMS.get('beta_1', 0.9),
        beta_2=TCN_PARAMS.get('beta_2', 0.999),
        epsilon=TCN_PARAMS.get('epsilon', 1e-07)
    )
    model.compile(optimizer=optimizer, loss='huber')
    return model

def prepare_data_for_training(X: np.ndarray, y: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """Prepare data for training, using all features for input but only prediction features for output."""
    X_train = X
    feature_indices = [FEATURE_COLUMNS.index(f) for f in PREDICTION_FEATURES]
    y_train = y[:, feature_indices]
    return X_train, y_train

def train_forecasting_model(model: Model, 
                          X_train: np.ndarray, 
                          y_train: np.ndarray,
                          X_val: np.ndarray,
                          y_val: np.ndarray,
                          model_name: str,
                          model_params: dict) -> Dict[str, Any]:
    """Train the forecasting model."""
    start_time = time.time()
    print_training_info(model_name, model, start_time)
    
    def lr_schedule(epoch, lr): 
        initial_lr = model_params.get('learning_rate', 0.001)
        if epoch < 10:
            return float(initial_lr)
        else:
            return float(lr * tf.math.exp(-0.1))

    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=model_params.get('patience',15), 
            restore_best_weights=True,
            mode='min'
        ),
        ModelCheckpoint(
            filepath=str(MODELS_DIR / f'{model_name}_best.keras'),
            monitor='val_loss',
            save_best_only=True,
            mode='min'
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5, 
            min_lr=1e-6, 
            mode='min',
            verbose=1
        ),
        TensorBoard(
            log_dir=str(RESULTS_DIR / 'logs' / model_name.lower()), 
            histogram_freq=TENSORBOARD_PARAMS.get('histogram_freq', 1),
            update_freq=TENSORBOARD_PARAMS.get('update_freq', 'epoch'),
            profile_batch=TENSORBOARD_PARAMS.get('profile_batch', 0) 
        )
    ]
    
    batch_size = model_params.get('batch_size', 64)
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))\
        .shuffle(buffer_size=len(X_train))\
        .batch(batch_size)\
        .prefetch(tf.data.AUTOTUNE)
    
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))\
        .batch(batch_size)\
        .prefetch(tf.data.AUTOTUNE)
    
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=model_params.get('epochs', 50),
        callbacks=callbacks,
        verbose=1 if PRINT_PARAMS['show_progress'] else 0
    )
    
    total_time = time.time() - start_time
    if PRINT_PARAMS['show_timing']:
        print(f"\nTraining for {model_name} completed in {total_time:.2f} seconds")
    
    return {
        'model': model,
        'history': history.history,
        'training_time': total_time
    }

def evaluate_forecasting_model(model: Model,
                             X_test: np.ndarray,
                             y_test: np.ndarray, 
                             model_name: str,
                             scaler=None,
                             timestamps_for_sequences=None) -> Dict[str, float]:
    """Evaluate the forecasting model."""
    eval_start_time = time.time()
    
    y_pred_scaled = model.predict(X_test, batch_size=LSTM_PARAMS.get('batch_size',64)) 
    
    mse = np.mean((y_test - y_pred_scaled) ** 2)
    mae = np.mean(np.abs(y_test - y_pred_scaled))
    rmse = np.sqrt(mse)
    
    if scaler is not None:
        y_test_full_shape = np.zeros((len(y_test), len(FEATURE_COLUMNS)))
        y_pred_full_shape = np.zeros((len(y_pred_scaled), len(FEATURE_COLUMNS)))
        
        feature_indices = [FEATURE_COLUMNS.index(f) for f in PREDICTION_FEATURES]
        
        y_test_full_shape[:, feature_indices] = y_test
        y_pred_full_shape[:, feature_indices] = y_pred_scaled
        
        y_test_denorm_full = scaler.inverse_transform(y_test_full_shape)
        y_pred_denorm_full = scaler.inverse_transform(y_pred_full_shape)
        
        y_test_denorm = y_test_denorm_full[:, feature_indices]
        y_pred_denorm = y_pred_denorm_full[:, feature_indices]
    else:
        y_test_denorm = y_test
        y_pred_denorm = y_pred_scaled
    
    plt.figure(figsize=(15, 4 * len(PREDICTION_FEATURES)))
    for i, feature_name_to_plot in enumerate(PREDICTION_FEATURES):
        plt.subplot(len(PREDICTION_FEATURES), 1, i + 1)
        plt.plot(y_test_denorm[:, i], label='Actual', alpha=0.7)
        plt.plot(y_pred_denorm[:, i], label='Predicted', alpha=0.7)
        plt.title(f'{feature_name_to_plot} - {model_name} Forecast')
        plt.legend()
        plt.grid(True)
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / f'{model_name.lower()}_forecast_vs_actual.png') 
    plt.close()
    
    plt.figure(figsize=(12, 6))
    errors_denorm = y_test_denorm - y_pred_denorm 
    for i, feature_name_to_plot in enumerate(PREDICTION_FEATURES):
        plt.subplot(1, len(PREDICTION_FEATURES), i + 1) 
        sns.histplot(errors_denorm[:, i], kde=True)
        plt.title(f'{feature_name_to_plot} Error Distribution (Denormalized)')
        plt.xlabel('Error')
        plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / f'{model_name.lower()}_error_distribution.png') 
    plt.close()
    
    last_sequence_scaled = X_test[-1:] 
    future_steps = FORECAST_HORIZON 
    future_predictions_scaled = []
    current_sequence_scaled = last_sequence_scaled.copy()

    for _ in range(future_steps):
        next_pred_scaled_for_pred_features = model.predict(current_sequence_scaled, verbose=0)[0] 
        future_predictions_scaled.append(next_pred_scaled_for_pred_features)
        
        new_time_step_scaled = current_sequence_scaled[0, -1, :].copy() 
        feature_indices_for_update = [FEATURE_COLUMNS.index(f) for f in PREDICTION_FEATURES]
        new_time_step_scaled[feature_indices_for_update] = next_pred_scaled_for_pred_features

        current_sequence_scaled = np.roll(current_sequence_scaled, -1, axis=1)
        current_sequence_scaled[0, -1, :] = new_time_step_scaled
    
    future_predictions_scaled = np.array(future_predictions_scaled)
    
    if scaler is not None:
        future_predictions_full_shape = np.zeros((len(future_predictions_scaled), len(FEATURE_COLUMNS)))
        # This line for feature_indices should be outside the loop if it's constant
        feature_indices = [FEATURE_COLUMNS.index(f) for f in PREDICTION_FEATURES] 
        future_predictions_full_shape[:, feature_indices] = future_predictions_scaled
        future_predictions_denorm_full = scaler.inverse_transform(future_predictions_full_shape)
        future_predictions_denorm = future_predictions_denorm_full[:, feature_indices]
    else:
        future_predictions_denorm = future_predictions_scaled
    
    future_plot_timestamps = None
    if timestamps_for_sequences is not None and len(timestamps_for_sequences) > 0:
        
        last_actual_timestamp = pd.to_datetime(timestamps_for_sequences[len(y_test)-1]) 
        future_plot_timestamps = pd.date_range(start=last_actual_timestamp + pd.Timedelta(minutes=1), periods=future_steps, freq='T')

    plt.figure(figsize=(15, 4 * len(PREDICTION_FEATURES)))
    for i, feature_name_to_plot in enumerate(PREDICTION_FEATURES):
        plt.subplot(len(PREDICTION_FEATURES), 1, i + 1)
        if future_plot_timestamps is not None:
            plt.plot(future_plot_timestamps, future_predictions_denorm[:, i], 
                     label='Future Predictions', alpha=0.7, linestyle='--', marker='o')
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
            plt.gca().xaxis.set_major_locator(mdates.MinuteLocator(interval=5 if future_steps > 10 else 1))
            plt.xticks(rotation=30, ha='right')
        else:
            plt.plot(future_predictions_denorm[:, i], 
                     label='Future Predictions', alpha=0.7, linestyle='--', marker='o')
        
        plt.title(f'{feature_name_to_plot} - {model_name} Future Forecast (Next {future_steps} Minutes)')
        plt.legend()
        plt.grid(True)
        plt.xlabel("Time")
        plt.ylabel(f"Predicted {feature_name_to_plot}")
    
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / f'{model_name.lower()}_future_forecast.png') 
    plt.close()
    
    if PRINT_PARAMS['show_metrics']:
        print(f"\n{model_name} Evaluation Metrics (on scaled test data):")
        print(f"MSE: {mse:.{PRINT_PARAMS['precision']}f}")
        print(f"MAE: {mae:.{PRINT_PARAMS['precision']}f}")
        print(f"RMSE: {rmse:.{PRINT_PARAMS['precision']}f}")
    
    if PRINT_PARAMS['show_timing']:
        eval_time = time.time() - eval_start_time
        print(f"Evaluation for {model_name} completed in {eval_time:.2f} seconds")
    
    np.save(RESULTS_DIR / f'{model_name.lower()}_test_predictions_denorm.npy', y_pred_denorm)
    np.save(RESULTS_DIR / f'{model_name.lower()}_future_predictions_denorm.npy', future_predictions_denorm)
    
    metrics_to_save = {
        'mse_scaled': float(mse),
        'mae_scaled': float(mae),
        'rmse_scaled': float(rmse)
    }
    if scaler is not None:
        metrics_to_save['mse_denorm'] = float(np.mean((y_test_denorm - y_pred_denorm) ** 2))
        metrics_to_save['mae_denorm'] = float(np.mean(np.abs(y_test_denorm - y_pred_denorm)))
        metrics_to_save['rmse_denorm'] = float(np.sqrt(metrics_to_save['mse_denorm']))

    with open(RESULTS_DIR / f'{model_name.lower()}_metrics.json', 'w') as f:
        json.dump(metrics_to_save, f, indent=4)
    
    return metrics_to_save

def run_forecasting():
    """Main function to run the forecasting phase."""
    print("\nLoading prepared data for forecasting...")
    start_time_main = time.time()
    
    X_train = np.load(DATA_DIR / 'X_train_forecast.npy')
    y_train_full = np.load(DATA_DIR / 'y_train_forecast.npy') 
    X_val = np.load(DATA_DIR / 'X_val_forecast.npy')
    y_val_full = np.load(DATA_DIR / 'y_val_forecast.npy')
    X_test = np.load(DATA_DIR / 'X_test_forecast.npy')
    y_test_full = np.load(DATA_DIR / 'y_test_forecast.npy')
    
    _, y_train = prepare_data_for_training(X_train, y_train_full)
    _, y_val = prepare_data_for_training(X_val, y_val_full)
    _, y_test = prepare_data_for_training(X_test, y_test_full)

    timestamps_for_test_sequences = None
    if (DATA_DIR / 'test_timestamps_for_sequences.npy').exists():
        timestamps_for_test_sequences = pd.to_datetime(np.load(DATA_DIR / 'test_timestamps_for_sequences.npy'))
        # Ensure timestamps align with the y_test data (which is shorter than original test_data due to sequencing)
        if len(timestamps_for_test_sequences) > len(y_test):
             timestamps_for_test_sequences = timestamps_for_test_sequences[:len(y_test)]
        print(f"Test sequence timestamps loaded successfully. Length: {len(timestamps_for_test_sequences)}")
    else:
        print("Warning: test_timestamps_for_sequences.npy not found.")
            
    scaler = None
    if (MODELS_DIR / 'scaler_forecast.joblib').exists():
        scaler = joblib.load(MODELS_DIR / 'scaler_forecast.joblib')
        print("Forecasting scaler loaded successfully.")
    else:
        print(f"Warning: scaler_forecast.joblib not found in {MODELS_DIR}. Evaluation will use scaled values for plots.")

    if PRINT_PARAMS['show_timing']:
        print(f"Data loaded in {time.time() - start_time_main:.2f} seconds")
    
    all_metrics = {}

    input_shape_common = (X_train.shape[1], X_train.shape[2])
    
    lstm_model = create_lstm_model(input_shape=input_shape_common)
    lstm_results = train_forecasting_model(lstm_model, X_train, y_train, X_val, y_val, 'LSTM', LSTM_PARAMS)
    all_metrics['lstm'] = evaluate_forecasting_model(lstm_results['model'], X_test, y_test, 'LSTM', scaler, timestamps_for_test_sequences)
    save_model(lstm_results['model'], 'lstm_forecasting_model') 

    tcn_model = create_tcn_model(input_shape=input_shape_common) 
    tcn_results = train_forecasting_model(tcn_model, X_train, y_train, X_val, y_val, 'TCN', TCN_PARAMS)
    all_metrics['tcn'] = evaluate_forecasting_model(tcn_results['model'], X_test, y_test, 'TCN', scaler, timestamps_for_test_sequences)
    save_model(tcn_results['model'], 'tcn_forecasting_model') 
    
    plt.figure(figsize=(15, 6))
    plt.subplot(1, 2, 1)
    plt.plot(lstm_results['history']['loss'], label='LSTM Train Loss')
    plt.plot(lstm_results['history']['val_loss'], label='LSTM Val Loss')
    plt.title('LSTM Training History')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (Huber)')
    plt.legend()
    plt.grid(True, alpha=0.5)
    
    plt.subplot(1, 2, 2)
    plt.plot(tcn_results['history']['loss'], label='TCN Train Loss')
    plt.plot(tcn_results['history']['val_loss'], label='TCN Val Loss')
    plt.title('TCN Training History')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (Huber)')
    plt.legend()
    plt.grid(True, alpha=0.5)
    
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / 'forecasting_training_history.png') 
    plt.close()
    
    metric_to_compare = 'mae_denorm' if 'mae_denorm' in all_metrics['lstm'] else 'mae_scaled'
    
    # Ensure keys exist before trying to access them for plotting
    metrics_for_plot = []
    if 'mse_denorm' in all_metrics['lstm'] and 'mae_denorm' in all_metrics['lstm'] and 'rmse_denorm' in all_metrics['lstm']:
        metrics_for_plot = ['mse_denorm', 'mae_denorm', 'rmse_denorm']
    elif 'mse_scaled' in all_metrics['lstm'] and 'mae_scaled' in all_metrics['lstm'] and 'rmse_scaled' in all_metrics['lstm']:
        metrics_for_plot = ['mse_scaled', 'mae_scaled', 'rmse_scaled']
    else:
        print("Warning: Not enough metrics found to generate comparison plot.")


    if metrics_for_plot: # Only plot if we have metrics
        lstm_plot_values = [all_metrics['lstm'].get(m, np.nan) for m in metrics_for_plot]
        tcn_plot_values = [all_metrics['tcn'].get(m, np.nan) for m in metrics_for_plot]

        x_labels = [m.replace('_denorm','').replace('_scaled','').upper() for m in metrics_for_plot]
        x_indices = np.arange(len(x_labels))
        bar_width = 0.35
        
        
        palette_name_from_config = PLOT_PARAMS.get('palette', 'deep')
        actual_color_palette = sns.color_palette(palette_name_from_config)

        fig_comp, ax_comp = plt.subplots(figsize=(12, 6))
        ax_comp.bar(x_indices - bar_width/2, lstm_plot_values, bar_width, label='LSTM', color=actual_color_palette[0])
        ax_comp.bar(x_indices + bar_width/2, tcn_plot_values, bar_width, label='TCN', color=actual_color_palette[1])
        
        ax_comp.set_xlabel('Metrics')
        ax_comp.set_ylabel('Value')
        ax_comp.set_title(f'Forecasting Model Comparison ({ "Denormalized" if "denorm" in metrics_for_plot[0] else "Scaled" })')
        ax_comp.set_xticks(x_indices)
        ax_comp.set_xticklabels(x_labels)
        ax_comp.legend()
        ax_comp.grid(True, axis='y', linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        plt.savefig(RESULTS_DIR / 'forecasting_model_comparison.png') 
        plt.close(fig_comp)

    if PRINT_PARAMS['show_timing']:
        print(f"\nTotal forecasting phase execution time: {time.time() - start_time_main:.2f} seconds")
    
    return {
        'lstm_metrics': all_metrics.get('lstm', {}), 
        'tcn_metrics': all_metrics.get('tcn', {}),
        'lstm_model': lstm_results.get('model') if 'lstm_results' in locals() else None, 
        'tcn_model': tcn_results.get('model') if 'tcn_results' in locals() else None,   
        'total_time': time.time() - start_time_main
    }

forecasting_run_results = run_forecasting()
print("\nForecasting models trained and evaluated.")

### Phase 2: Anomaly Detection with LSTM Autoencoder

This section focuses on training and evaluating an LSTM-based Autoencoder specifically for anomaly detection. The Autoencoder is trained only on "normal" data sequences. The underlying principle is that the model will learn to reconstruct normal patterns with low error, while anomalous patterns will result in higher reconstruction errors, allowing for their identification.

**What this cell does:**
1.  **Setup & Utility Functions (Defined within this cell for self-containment regarding plotting):**
    *   Sets Matplotlib/Seaborn plotting styles and TensorFlow mixed precision policy.
    *   **`plot_confusion_matrix()`**, **`plot_roc_curve()`**, **`plot_precision_recall_curve()`**, **`plot_anomaly_scores_distribution()`**: These are local copies of the plotting utilities specifically for visualizing the Autoencoder's anomaly detection performance. They generate and save relevant plots to the `RESULTS_DIR`.
2.  **Define Autoencoder-specific Helper Functions:**
    *   `print_training_info_ad()`: Similar to the forecasting version, but for the Autoencoder training.
    *   `create_autoencoder()`: Defines the architecture of the LSTM-based Autoencoder. It consists of an encoder path with Bidirectional LSTM layers compressing the input sequence into a bottleneck representation, and a decoder path with Bidirectional LSTM layers attempting to reconstruct the original sequence from this representation. It uses Adam optimizer and MSE loss.
    *   `plot_training_history_ad()`: Plots the training and validation loss (MSE) and MAE for the Autoencoder over epochs.
    *   `train_autoencoder()`:
        *   Manages the training process for the Autoencoder, using callbacks like `EarlyStopping`, `ModelCheckpoint`, and `ReduceLROnPlateau`.
        *   Trains the model using normal data sequences from the training set (`X_train_ad`) as both input and target.
        *   Uses a validation set (`X_val_ad_mixed`, also using its inputs as targets for reconstruction loss) to monitor for overfitting.
    *   `detect_anomalies_autoencoder()`:
        *   Calculates reconstruction errors (Mean Squared Error) for the validation set (`X_val_for_thresh`).
        *   Performs a threshold analysis: iterates through various percentile-based thresholds derived from validation errors and calculates Precision, Recall, and F1-score against the true validation labels (`y_val_for_thresh_labels`).
        *   Saves this threshold analysis to `autoencoder_threshold_analysis.csv`.
        *   Selects the optimal threshold based on maximizing the specified `optimization_metric` (e.g., F1-score) on the validation set.
        *   Calculates reconstruction errors for the test set (`X_test`).
        *   Classifies test instances as anomalous if their reconstruction error exceeds the chosen optimal threshold.
    *   `evaluate_anomaly_detection_ad()`:
        *   Takes true labels, predicted binary anomaly labels, and raw anomaly scores for the test set.
        *   Calculates and prints a detailed classification report (Precision, Recall, F1-score for normal and anomaly classes).
        *   Calls the local plotting functions (`plot_confusion_matrix`, `plot_roc_curve`, `plot_precision_recall_curve`, `plot_anomaly_scores_distribution`) to visualize test set performance.
        *   Saves the performance metrics to `all_anomaly_metrics.csv` (appending/overwriting for the "Autoencoder" model) and a model-specific JSON file.
3.  **`run_anomaly_detection()` Function:**
    *   Loads the preprocessed data specifically prepared for the Autoencoder (`X_train_ad`, `X_val_ad_mixed`, `y_val_ad_labels`, `X_test_ad_mixed`, `y_test_ad_labels`).
    *   Instantiates, trains (using `train_autoencoder`), and then uses the trained Autoencoder to detect anomalies on the test set (using `detect_anomalies_autoencoder`).
    *   Evaluates the anomaly detection performance on the test set (using `evaluate_anomaly_detection_ad`).
    *   Saves the trained Autoencoder model.
4.  **Execution:** Calls `run_anomaly_detection()` to perform all the Autoencoder-related steps.

**Why these choices were made:**
*   **LSTM Autoencoder:** A common and effective deep learning approach for unsupervised/semi-supervised anomaly detection in sequential data. It learns a compressed representation of normal data.
*   **Training on Normal Data Only:** This is a core principle for this type of anomaly detection. The model should only learn what "normal" looks like.
*   **Reconstruction Error as Anomaly Score:** Deviations from learned normal patterns result in higher reconstruction errors, which serve as the anomaly score.
*   **Threshold Optimization on Validation Set:** Crucial for converting continuous anomaly scores into binary anomaly/normal classifications. Using the validation set prevents data leakage from the test set into the threshold selection process and aims for a threshold that generalizes well. Maximizing F1-score is a common strategy to balance precision and recall.
*   **Comprehensive Evaluation:** Using Precision, Recall, F1-score, AUC-ROC, AUC-PR, and confusion matrices provides a thorough assessment of the anomaly detection performance.

**Expected Output:**
*   Print statements indicating data loading and model training stages.
*   Autoencoder model summary.
*   Training progress (epochs, loss, validation loss, MAE, validation MAE).
*   Results of the threshold analysis on the validation set printed as a table.
*   The chosen optimal threshold and its corresponding F1-score (or other optimization metric).
*   Classification report for the test set performance.
*   Saved plot files in the `results/` directory:
    *   `Autoencoder_Training_History.png`
    *   `Autoencoder_Anomaly_Scores_Distribution.png`
    *   `Autoencoder_Confusion_Matrix.png`
    *   `Autoencoder_ROC_Curve.png`
    *   `Autoencoder_Precision_Recall_Curve.png`
    *   (Possibly `Autoencoder_Test_Set_Threshold_Analysis.png` if `plot_anomaly_results` from utils was called with that specific name)
*   Saved `autoencoder_threshold_analysis.csv`.
*   Saved `all_anomaly_metrics.csv` (updated or created).
*   Saved `autoencoder_metrics_ad.json`.
*   Saved the Autoencoder model (`autoencoder_anomaly_model.keras`) in the `models/` directory.
*   A final message "Autoencoder Anomaly Detection phase completed!"
*   The cell will return a dictionary `autoencoder_evaluation_metrics` containing key performance indicators.

# Phase2 Anomaly Detection

In [ ]:
# Cell 5: Anomaly Detection Model (Autoencoder) Training and Evaluation
import time
from datetime import datetime
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score
import json
import pandas as pd
from pathlib import Path 


def plot_confusion_matrix(y_true: np.ndarray, y_pred: np.ndarray, model_name: str):
    """Plot confusion matrix for anomaly detection results."""
    
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{model_name} - Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(RESULTS_DIR / f'{model_name}_Confusion_Matrix.png')
    plt.close()

def plot_roc_curve(y_true: np.ndarray, scores: np.ndarray, model_name: str):
    """Plot ROC curve for anomaly detection results."""
    
    fpr, tpr, _ = roc_curve(y_true, scores)
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.3f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'{model_name} - ROC Curve')
    plt.legend(loc="lower right")
    plt.savefig(RESULTS_DIR / f'{model_name}_ROC_Curve.png')
    plt.close()

def plot_precision_recall_curve(y_true: np.ndarray, scores: np.ndarray, model_name: str):
    """Plot precision-recall curve for anomaly detection results."""
   
    precision_curve_vals, recall_curve_vals, _ = precision_recall_curve(y_true, scores) # Renamed to avoid conflict
    avg_precision = average_precision_score(y_true, scores)
    plt.figure(figsize=(8, 6))
    plt.plot(recall_curve_vals, precision_curve_vals, label=f'PR curve (AP = {avg_precision:.3f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'{model_name} - Precision-Recall Curve')
    plt.legend(loc="lower left")
    plt.savefig(RESULTS_DIR / f'{model_name}_Precision_Recall_Curve.png')
    plt.close()

def plot_anomaly_scores_distribution(y_true: np.ndarray, scores: np.ndarray, model_name: str):
    """Plot distribution of anomaly scores by class with enhanced visualization."""
    
    plt.figure(figsize=(15, 10))
    
    df_plot = pd.DataFrame({ 
        'Score': scores,
        'Class': ['Anomaly' if y == 1 else 'Normal' for y in y_true]
    })
    
    gs = plt.GridSpec(2, 1, height_ratios=[3, 1])
    ax1 = plt.subplot(gs[0])
    sns.histplot(data=df_plot, x='Score', hue='Class', kde=True, 
                palette={'Normal': 'blue', 'Anomaly': 'red'},
                alpha=0.6, bins=50, ax=ax1) 
    
    for cls_name in ['Normal', 'Anomaly']:
        mean_score = df_plot[df_plot['Class'] == cls_name]['Score'].mean()
        if not pd.isna(mean_score): 
            ax1.axvline(x=mean_score, color='red' if cls_name == 'Anomaly' else 'blue',
                       linestyle='--', alpha=0.8,
                       label=f'{cls_name} Mean: {mean_score:.4f}')
    
    ax1.set_title(f'{model_name} - Anomaly Scores Distribution', pad=20)
    ax1.set_xlabel('Anomaly Score')
    ax1.set_ylabel('Count')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    ax2 = plt.subplot(gs[1])
    sns.boxplot(data=df_plot, x='Class', y='Score', palette={'Normal': 'blue', 'Anomaly': 'red'}, ax=ax2) # Pass ax
    ax2.set_title('Score Distribution by Class')
    ax2.grid(True, alpha=0.3)
    
    stats_text = f"""
    Statistics:
    Normal samples: {len(df_plot[df_plot['Class'] == 'Normal'])}
    Anomaly samples: {len(df_plot[df_plot['Class'] == 'Anomaly'])}
    Normal mean: {df_plot[df_plot['Class'] == 'Normal']['Score'].mean():.4f}
    Anomaly mean: {df_plot[df_plot['Class'] == 'Anomaly']['Score'].mean():.4f}
    Normal std: {df_plot[df_plot['Class'] == 'Normal']['Score'].std():.4f}
    Anomaly std: {df_plot[df_plot['Class'] == 'Anomaly']['Score'].std():.4f}
    """
    plt.figtext(0.02, 0.02, stats_text, fontsize=10, bbox=dict(facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / f'{model_name}_Anomaly_Scores_Distribution.png', dpi=300, bbox_inches='tight')
    plt.close()


plt.rcParams.update(PLOT_PARAMS['rc'])
sns.set_context(PLOT_PARAMS['context'])


tf.keras.mixed_precision.set_global_policy('mixed_float16') 

def print_training_info_ad(model_name: str, model: Model, start_time: float): 
    """Print training information."""
    if PRINT_PARAMS['show_summary']:
        print(f"\n{'='*50}")
        print(f"Training {model_name}")
        print(f"{'='*50}")
        if model is not None:
            model.summary(line_length=120)
    
    if PRINT_PARAMS['show_timing']:
        elapsed_time = time.time() - start_time
        print(f"\nTraining started at: {datetime.fromtimestamp(start_time).strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Elapsed time: {elapsed_time:.2f} seconds")

def create_autoencoder(input_shape: Tuple[int, int]) -> Model:
    """Create an improved autoencoder model for anomaly detection."""
    encoder_input = Input(shape=input_shape)
    x = Bidirectional(LSTM(128, return_sequences=True, dropout=AUTOENCODER_PARAMS.get('dropout', 0.2)))(encoder_input)
    x = BatchNormalization()(x)
    x = Bidirectional(LSTM(64, return_sequences=False, dropout=AUTOENCODER_PARAMS.get('dropout', 0.2)))(x)
    x = BatchNormalization()(x)
    encoded = Dense(AUTOENCODER_PARAMS.get('units', [64,32,16])[-1], activation='relu')(x) 
    encoded = BatchNormalization()(encoded)
    
    x = Dense(64, activation='relu')(encoded) 
    x = BatchNormalization()(x)
    x = RepeatVector(input_shape[0])(x) 
    x = Bidirectional(LSTM(64, return_sequences=True, dropout=AUTOENCODER_PARAMS.get('dropout', 0.2)))(x)
    x = BatchNormalization()(x)
    x = Bidirectional(LSTM(128, return_sequences=True, dropout=AUTOENCODER_PARAMS.get('dropout', 0.2)))(x)
    x = BatchNormalization()(x)
    decoded = TimeDistributed(Dense(input_shape[1], activation='linear'))(x) 
    
    autoencoder = Model(encoder_input, decoded)
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=AUTOENCODER_PARAMS.get('learning_rate', 0.001),
        beta_1=AUTOENCODER_PARAMS.get('beta_1', 0.9),
        beta_2=AUTOENCODER_PARAMS.get('beta_2', 0.999),
        epsilon=AUTOENCODER_PARAMS.get('epsilon', 1e-07)
    )
    autoencoder.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return autoencoder

def plot_training_history_ad(history: Dict[str, Any], model_name: str): 
    """Plot training history of the autoencoder model."""
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history['loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title(f'{model_name} - Loss (MSE)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.5)
    
    plt.subplot(1, 2, 2)
    plt.plot(history['mae'], label='Training MAE')
    plt.plot(history['val_mae'], label='Validation MAE')
    plt.title(f'{model_name} - MAE')
    plt.xlabel('Epoch')
    plt.ylabel('MAE')
    plt.legend()
    plt.grid(True, alpha=0.5)
    
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / f'{model_name}_Training_History.png') 
    plt.close()

def train_autoencoder(model: Model, X_train: np.ndarray, X_val: np.ndarray) -> Dict[str, Any]:
    """Train the autoencoder model with improved settings."""
    start_time = time.time()
    print_training_info_ad("Autoencoder", model, start_time)
    
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=AUTOENCODER_PARAMS.get('patience', 10), 
            restore_best_weights=True,
            mode='min'
        ),
        ModelCheckpoint(
            filepath=str(MODELS_DIR / 'autoencoder_best.keras'),
            monitor='val_loss',
            save_best_only=True,
            mode='min'
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5, 
            patience=5,  
            min_lr=1e-6, 
            verbose=1
        ),
        TensorBoard( 
            log_dir=str(RESULTS_DIR / 'logs' / 'autoencoder'),
            histogram_freq=TENSORBOARD_PARAMS.get('histogram_freq',1),
            update_freq=TENSORBOARD_PARAMS.get('update_freq','epoch'),
            profile_batch=TENSORBOARD_PARAMS.get('profile_batch',0)
        )
    ]
    
    batch_size = AUTOENCODER_PARAMS.get('batch_size', 64)
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, X_train))\
        .shuffle(buffer_size=len(X_train))\
        .batch(batch_size)\
        .prefetch(tf.data.AUTOTUNE)
    
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, X_val))\
        .batch(batch_size)\
        .prefetch(tf.data.AUTOTUNE)
    
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=AUTOENCODER_PARAMS.get('epochs', 50),
        callbacks=callbacks,
        verbose=1 if PRINT_PARAMS['show_progress'] else 0
    )
    
    plot_training_history_ad(history.history, "Autoencoder") 
    
    total_time = time.time() - start_time
    if PRINT_PARAMS['show_timing']:
        print(f"\nAutoencoder training completed in {total_time:.2f} seconds")
    
    return {
        'model': model,
        'history': history.history,
        'training_time': total_time
    }

def detect_anomalies_autoencoder(model: Model, 
                               X_test: np.ndarray, 
                               X_val_for_thresh: np.ndarray, 
                               y_val_for_thresh_labels: np.ndarray, 
                               threshold_percentiles_list: List[float], 
                               optimization_metric: str = 'f1' 
                               ) -> Tuple[np.ndarray, np.ndarray, float]: 
    """Detect anomalies using the autoencoder model, with threshold optimized on validation data."""
    
    print("\nCalculating reconstruction errors on validation set for threshold optimization...")
    val_predictions = model.predict(X_val_for_thresh, batch_size=AUTOENCODER_PARAMS.get('batch_size', 256), verbose=0)
    val_reconstruction_errors = np.mean(np.square(X_val_for_thresh - val_predictions), axis=(1, 2))
    
    y_val_binary_labels = (np.any(y_val_for_thresh_labels > 0, axis=1)).astype(int) if len(y_val_for_thresh_labels.shape) > 1 and y_val_for_thresh_labels.shape[1] > 1 else (y_val_for_thresh_labels > 0.5).astype(int)
    
    if len(y_val_binary_labels) != len(val_reconstruction_errors):
         raise ValueError(f"Length mismatch: y_val_binary_labels ({len(y_val_binary_labels)}) vs val_reconstruction_errors ({len(val_reconstruction_errors)})")

    print("\nAnalyzing different thresholds for Autoencoder on validation data:")
    threshold_analysis_results = []
    candidate_thresholds = np.percentile(val_reconstruction_errors, threshold_percentiles_list)
    
    for thresh_val in candidate_thresholds:
        y_pred_val_binary = (val_reconstruction_errors > thresh_val).astype(int)
        p = precision_score(y_val_binary_labels, y_pred_val_binary, zero_division=0)
        r = recall_score(y_val_binary_labels, y_pred_val_binary, zero_division=0)
        f1 = f1_score(y_val_binary_labels, y_pred_val_binary, zero_division=0)
        cm_thresh = confusion_matrix(y_val_binary_labels, y_pred_val_binary)
        fp_thresh = cm_thresh[0,1] if cm_thresh.shape == (2,2) and len(cm_thresh[0]) > 1 else 0
        fn_thresh = cm_thresh[1,0] if cm_thresh.shape == (2,2) and len(cm_thresh) > 1 else 0
        
        threshold_analysis_results.append({
            'Threshold': thresh_val, 'Precision': p, 'Recall': r, 'F1': f1, 'FP': fp_thresh, 'FN': fn_thresh
        })
    
    results_df = pd.DataFrame(threshold_analysis_results)
    print("\nValidation Set Threshold Analysis Results:")
    print(results_df)
    results_df.to_csv(RESULTS_DIR / "autoencoder_threshold_analysis.csv", index=False)
    
    if optimization_metric == 'f1' and 'F1' in results_df.columns and not results_df['F1'].empty:
        best_row = results_df.loc[results_df['F1'].idxmax()]
    elif optimization_metric == 'precision' and 'Precision' in results_df.columns and not results_df['Precision'].empty:
        best_row = results_df.sort_values(by=['Precision', 'Recall'], ascending=[False, False]).iloc[0]
    elif optimization_metric == 'recall' and 'Recall' in results_df.columns and not results_df['Recall'].empty:
        best_row = results_df.sort_values(by=['Recall', 'Precision'], ascending=[False, False]).iloc[0]
    elif 'F1' in results_df.columns and not results_df['F1'].empty: # Fallback to F1 if metric not found or empty
        best_row = results_df.loc[results_df['F1'].idxmax()]
    else: # Absolute fallback if F1 is also problematic
        print("Warning: Could not determine best threshold based on optimization metric. Using median error as threshold.")
        best_row = pd.Series({'Threshold': np.median(val_reconstruction_errors), 'Precision': np.nan, 'Recall': np.nan, 'F1': np.nan})


    chosen_threshold = best_row['Threshold']
    print(f"\nChosen threshold based on '{optimization_metric}' on validation set: {chosen_threshold:.4f} (P: {best_row['Precision']:.3f}, R: {best_row['Recall']:.3f}, F1: {best_row['F1']:.3f})")
    
    print("\nCalculating reconstruction errors on test set...")
    test_predictions = model.predict(X_test, batch_size=AUTOENCODER_PARAMS.get('batch_size',256), verbose=0)
    test_reconstruction_errors = np.mean(np.square(X_test - test_predictions), axis=(1, 2))
    
    test_anomalies_predicted = (test_reconstruction_errors > chosen_threshold).astype(int)
    
    return test_anomalies_predicted, test_reconstruction_errors, chosen_threshold


def evaluate_anomaly_detection_ad(y_true_labels: np.ndarray, 
                               y_pred_labels: np.ndarray,
                               anomaly_scores_for_roc_pr: np.ndarray,
                               model_name: str,
                               chosen_threshold_for_plotting: float) -> Dict[str, float]:
    """Evaluate anomaly detection model performance."""
    y_true_binary = (np.any(y_true_labels > 0, axis=1)).astype(int) if len(y_true_labels.shape) > 1 and y_true_labels.shape[1] > 1 else (y_true_labels > 0.5).astype(int)
    
    if len(y_pred_labels) != len(y_true_binary):
        raise ValueError(f"Length mismatch in evaluate_anomaly_detection_ad: y_true_binary ({len(y_true_binary)}) vs y_pred_labels ({len(y_pred_labels)})")
    if len(anomaly_scores_for_roc_pr) != len(y_true_binary):
         raise ValueError(f"Length mismatch: y_true_binary ({len(y_true_binary)}) vs anomaly_scores_for_roc_pr ({len(anomaly_scores_for_roc_pr)})")

    precision = precision_score(y_true_binary, y_pred_labels, zero_division=0)
    recall = recall_score(y_true_binary, y_pred_labels, zero_division=0)
    f1 = f1_score(y_true_binary, y_pred_labels, zero_division=0)
    
    roc_auc_val = np.nan
    avg_precision_val = np.nan

    if len(np.unique(y_true_binary)) > 1:
        try:
            roc_auc_val = roc_auc_score(y_true_binary, anomaly_scores_for_roc_pr)
            avg_precision_val = average_precision_score(y_true_binary, anomaly_scores_for_roc_pr)
        except ValueError as e:
            print(f"Warning for {model_name}: Could not calculate AUC/AP: {e}")
    else:
        print(f"Warning for {model_name}: Only one class present in true labels. AUC/AP cannot be computed meaningfully.")

    print(f"\n{model_name} Classification Report (Test Set):")
    print(classification_report(y_true_binary, y_pred_labels, zero_division=0, target_names=['Normal', 'Anomaly']))
    
    
    plot_confusion_matrix(y_true_binary, y_pred_labels, model_name) 
    if not np.isnan(roc_auc_val):
        plot_roc_curve(y_true_binary, anomaly_scores_for_roc_pr, model_name) 
    if not np.isnan(avg_precision_val):
        plot_precision_recall_curve(y_true_binary, anomaly_scores_for_roc_pr, model_name)
    
    plot_anomaly_scores_distribution(y_true_binary, anomaly_scores_for_roc_pr, model_name)

    metrics_dict = {
        'precision': float(precision), 'recall': float(recall), 'f1_score': float(f1),
        'auc_score': float(roc_auc_val) if not np.isnan(roc_auc_val) else None, 
        'average_precision': float(avg_precision_val) if not np.isnan(avg_precision_val) else None
    }
    
    model_metrics_df = pd.DataFrame([metrics_dict])
    model_metrics_df.insert(0, 'Model', model_name)
    
    main_metrics_file = RESULTS_DIR / 'all_anomaly_metrics.csv' 
    if main_metrics_file.exists():
        existing_metrics_df = pd.read_csv(main_metrics_file)
        existing_metrics_df = existing_metrics_df[existing_metrics_df['Model'] != model_name] 
        combined_metrics_df = pd.concat([existing_metrics_df, model_metrics_df], ignore_index=True)
    else:
        combined_metrics_df = model_metrics_df
    combined_metrics_df.to_csv(main_metrics_file, index=False)

    with open(RESULTS_DIR / f'{model_name.lower()}_metrics_ad.json', 'w') as f: 
        json.dump(metrics_dict, f, indent=4)
    
    return metrics_dict

def run_anomaly_detection():
    """Run anomaly detection with Autoencoder."""
    print("\nStarting Autoencoder Anomaly Detection Phase...")
    start_time_main = time.time()

    X_train_ad = np.load(DATA_DIR / 'X_train_ad.npy') 
    X_val_ad_mixed = np.load(DATA_DIR / 'X_val_ad.npy') 
    y_val_ad_labels = np.load(DATA_DIR / 'y_val_ad_labels.npy') 
    X_test_ad_mixed = np.load(DATA_DIR / 'X_test_ad.npy') 
    y_test_ad_labels = np.load(DATA_DIR / 'y_test_ad_labels.npy') 

    print("\nData shapes for Autoencoder:")
    print(f"X_train_ad (normal only): {X_train_ad.shape}")
    print(f"X_val_ad (mixed): {X_val_ad_mixed.shape}, y_val_ad_labels: {y_val_ad_labels.shape}")
    print(f"X_test_ad (mixed): {X_test_ad_mixed.shape}, y_test_ad_labels: {y_test_ad_labels.shape}")

    print("\nTraining Autoencoder...")
    input_shape_ae = (X_train_ad.shape[1], X_train_ad.shape[2])
    autoencoder = create_autoencoder(input_shape=input_shape_ae)
    autoencoder_results = train_autoencoder(autoencoder, X_train_ad, X_val_ad_mixed) 
    
    print("\nDetecting anomalies with Autoencoder...")
    predicted_anomalies_test, test_anomaly_scores, chosen_threshold_from_val = detect_anomalies_autoencoder(
        autoencoder_results['model'],
        X_test_ad_mixed,
        X_val_ad_mixed, 
        y_val_ad_labels, 
        threshold_percentiles_list=AUTOENCODER_PARAMS['threshold_analysis_percentiles'],
        optimization_metric=AUTOENCODER_PARAMS['threshold_optimization_metric']
    )
    
    print("\nEvaluating Autoencoder on Test Set...")
    autoencoder_test_metrics = evaluate_anomaly_detection_ad(
        y_test_ad_labels, predicted_anomalies_test, test_anomaly_scores, "Autoencoder", chosen_threshold_from_val
    )
    
    # Assuming save_model is globally available (from Cell 2 or imported)
    save_model(autoencoder_results['model'], 'autoencoder_anomaly_model')
    
    if PRINT_PARAMS['show_timing']:
        print(f"\nTotal Autoencoder anomaly detection phase execution time: {time.time() - start_time_main:.2f} seconds")
    
    print("\nAutoencoder Anomaly Detection phase completed!")
    return autoencoder_test_metrics


autoencoder_evaluation_metrics = run_anomaly_detection()
print("\nAutoencoder trained and evaluated.")

### Phase 3: Final Model Evaluation and Comparison

This final phase consolidates the results from the forecasting and anomaly detection models, provides comparative visualizations, and generates a summary report.

**What this cell does:**
1.  **Define Local Plotting Utilities (if needed, or rely on Cell 2):**
    *   `plot_confusion_matrix_eval()` and `plot_roc_curve_eval()`: These are defined locally within this cell (as per the provided script) to potentially customize evaluation-phase plots or avoid conflicts if utility functions in Cell 2 were modified. In a fully streamlined notebook, these would ideally be consolidated with the utilities in Cell 2.
2.  **Define Comparison and Reporting Functions:**
    *   `compare_forecasting_metrics_dfs()`: Loads forecasting metrics from JSON files (one per model, e.g., `lstm_metrics.json`, `tcn_metrics.json`), creates a comparison DataFrame, plots a bar chart of key metrics (MSE, MAE, RMSE, prioritizing denormalized versions), and saves the plot.
    *   `compare_anomaly_detection_metrics_dfs()`: Loads anomaly detection metrics from JSON files (e.g., `autoencoder_anomaly_model_metrics_ad.json`), creates a comparison DataFrame, plots a bar chart (Precision, Recall, F1, AUC, Avg. Precision), and saves it. *Alternatively, it's set up to load from `all_anomaly_metrics.csv` if available, which is a more consolidated source.*
    *   `load_all_data_for_evaluation()`: Loads necessary test data arrays (`X_test_ad`, `y_test_ad_labels`), forecasting predictions, and recalculates/loads Autoencoder anomaly scores and predicted labels for the test set using the previously determined optimal threshold. This ensures all data needed for final evaluation plots and summaries is available.
    *   `generate_final_summary_report()`: Takes the comparison DataFrames from forecasting and anomaly detection. It identifies the best performing model for each task based on a primary metric (e.g., MAE for forecasting, F1-score for anomaly detection). It also compiles key insights and general recommendations for improvement.
    *   `save_evaluation_results()`: (This function was in the original `evaluate_models.py` but its functionality is largely covered by individual saving actions in other functions and the `generate_final_summary_report` saving its own output). For this notebook cell, the main output saving is the `final_evaluation_summary_report.json`.
3.  **`run_evaluation_phase()` Function:**
    *   Orchestrates the final evaluation.
    *   Calls `load_all_data_for_evaluation()` to get test data and model outputs.
    *   Calls `compare_forecasting_metrics_dfs()` to compare LSTM and TCN.
    *   Calls `compare_anomaly_detection_metrics_dfs()` to display (and potentially re-plot) anomaly detection model metrics (primarily Autoencoder in this setup).
    *   Calls `generate_final_summary_report()` to create a JSON summary.
    *   Prints key findings and recommendations from the summary report.
4.  **Execution:** Calls `run_evaluation_phase()` to perform all comparison and reporting steps.

**Why these steps are performed:**
*   **Consolidated View:** To bring together results from different modeling phases for a holistic understanding of the project's outcomes.
*   **Model Benchmarking:** To quantitatively and visually compare the performance of different approaches (LSTM vs. TCN for forecasting; and potentially different anomaly detectors if more were added).
*   **Actionable Insights:** To derive conclusions about which models perform best for specific tasks and to identify areas for future improvement.
*   **Reporting:** To generate a structured summary (JSON file) that encapsulates the main findings, suitable for documentation or sharing.

**Expected Output:**
*   Print statements indicating the start of the evaluation phase and data loading.
*   If forecasting metric files exist:
    *   A Pandas DataFrame comparing LSTM and TCN forecasting metrics.
    *   A saved plot `final_forecasting_models_comparison.png`.
*   If anomaly detection metric files/CSVs exist:
    *   A Pandas DataFrame comparing anomaly detection model(s) metrics.
    *   A saved plot `final_anomaly_detection_models_comparison.png`.
*   Key insights and recommendations printed from the generated summary report.
*   A `final_evaluation_summary_report.json` file saved in the `RESULTS_DIR`.
*   A final message indicating the completion of the evaluation phase.
*   The cell will return the `summary_report` dictionary.

# Phase3 Evaluation

In [ ]:
# Cell 6: Model Evaluation and Comparison

def plot_confusion_matrix_eval(y_true: np.ndarray, 
                         y_pred: np.ndarray,
                         model_name: str):
    """Plot confusion matrix for anomaly detection results."""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / f'eval_confusion_matrix_{model_name.lower().replace(" ", "_")}.png')
    plt.close()

def plot_roc_curve_eval(y_true: np.ndarray, 
                  y_score: np.ndarray,
                  model_name: str):
    """Plot ROC curve for anomaly detection results."""
    fpr, tpr, _ = roc_curve(y_true, y_score)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2,
             label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - {model_name}')
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / f'eval_roc_curve_{model_name.lower().replace(" ", "_")}.png')
    plt.close()


def compare_forecasting_metrics_dfs(forecasting_metrics_files: Dict[str, Path]) -> pd.DataFrame:
    """Load and compare forecasting model metrics from JSON files."""
    all_metrics_list = []
    for model_name, file_path in forecasting_metrics_files.items():
        if file_path.exists():
            with open(file_path, 'r') as f:
                metrics = json.load(f)
            metrics['model'] = model_name
            all_metrics_list.append(metrics)
        else:
            print(f"Warning: Metrics file not found for {model_name} at {file_path}")
    
    if not all_metrics_list:
        return pd.DataFrame()

    metrics_df = pd.DataFrame(all_metrics_list)
    metrics_df = metrics_df.set_index('model')
    
    
    plot_cols = [col for col in ['mae_denorm', 'rmse_denorm', 'mse_denorm'] if col in metrics_df.columns]
    if not plot_cols:
        plot_cols = [col for col in ['mae_scaled', 'rmse_scaled', 'mse_scaled'] if col in metrics_df.columns]

    if plot_cols:
        metrics_df[plot_cols].plot(kind='bar', figsize=(12, 7))
        plt.title('Forecasting Models Comparison')
        plt.ylabel('Metric Value')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(RESULTS_DIR / 'final_forecasting_models_comparison.png')
        plt.close()
    else:
        print("No suitable metrics found for plotting forecasting comparison.")
        
    return metrics_df


def compare_anomaly_detection_metrics_dfs(anomaly_metrics_files: Dict[str, Path]) -> pd.DataFrame:
    """Load and compare anomaly detection model metrics from JSON files."""
    all_metrics_list = []
    for model_name, file_path in anomaly_metrics_files.items():
        if file_path.exists():
            with open(file_path, 'r') as f:
                metrics = json.load(f)
            metrics['model'] = model_name
            all_metrics_list.append(metrics)
        else:
            print(f"Warning: Metrics file not found for {model_name} at {file_path}")

    if not all_metrics_list:
        return pd.DataFrame()
        
    metrics_df = pd.DataFrame(all_metrics_list)
    metrics_df = metrics_df.set_index('model')
    
    plot_cols = ['precision', 'recall', 'f1_score', 'auc_score', 'average_precision']
    metrics_df[plot_cols].plot(kind='bar', figsize=(12, 7))
    plt.title('Anomaly Detection Models Comparison')
    plt.ylabel('Metric Value')
    plt.xticks(rotation=45, ha='right')
    plt.ylim(0, 1.05) # Metrics are typically between 0 and 1
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / 'final_anomaly_detection_models_comparison.png')
    plt.close()

    return metrics_df

def load_all_data_for_evaluation() -> Dict[str, Any]:
    """Load all necessary data for the final evaluation phase."""
    data = {}
    try:
        print("Loading test data for final evaluation...")
        data['X_test_ad'] = np.load(DATA_DIR / 'X_test_ad.npy')
        data['y_test_ad_labels'] = np.load(DATA_DIR / 'y_test_ad_labels.npy')
        # For forecasting evaluation, if needed, load y_test_forecast
        data['y_test_forecast_pred_features'] = np.load(DATA_DIR / 'y_test_forecast.npy')[:, [FEATURE_COLUMNS.index(f) for f in PREDICTION_FEATURES]]

        if (DATA_DIR / 'test_timestamps_for_sequences.npy').exists():
            data['test_timestamps_for_sequences'] = pd.to_datetime(np.load(DATA_DIR / 'test_timestamps_for_sequences.npy'))
        else:
            data['test_timestamps_for_sequences'] = None
            print("Warning: test_timestamps_for_sequences.npy not found.")
        print("Test data loaded.")
    except Exception as e:
        print(f"Error loading base test data: {e}")
        
        return {'error': 'Failed to load essential test data.'}

    # Load model predictions and scores
    try:
        print("Loading model predictions and scores...")
        data['lstm_predictions_denorm'] = np.load(RESULTS_DIR / 'lstm_test_predictions_denorm.npy')
        data['tcn_predictions_denorm'] = np.load(RESULTS_DIR / 'tcn_test_predictions_denorm.npy')
        
        
        autoencoder_threshold_df = pd.read_csv(RESULTS_DIR / "autoencoder_threshold_analysis.csv")
        best_f1_row = autoencoder_threshold_df.loc[autoencoder_threshold_df[AUTOENCODER_PARAMS['threshold_optimization_metric']].idxmax()]
        data['autoencoder_chosen_threshold'] = best_f1_row['Threshold']
        
        
        ae_model = load_model('autoencoder_anomaly_model') 
        ae_test_predictions = ae_model.predict(data['X_test_ad'])
        data['autoencoder_test_scores'] = np.mean(np.square(data['X_test_ad'] - ae_test_predictions), axis=(1,2))
        data['autoencoder_test_predicted_labels'] = (data['autoencoder_test_scores'] > data['autoencoder_chosen_threshold']).astype(int)
        print("Model predictions and scores loaded/recalculated.")

    except Exception as e:
        print(f"Error loading model outputs: {e}")
        data['load_output_error'] = str(e)
        
    return data

def generate_final_summary_report(forecast_comparison_df: pd.DataFrame, anomaly_comparison_df: pd.DataFrame) -> Dict[str, Any]:
    """Generate a comprehensive summary report of all model performances."""
    report = {
        'timestamp': datetime.now().isoformat(),
        'forecasting_summary': None,
        'anomaly_detection_summary': None,
        'overall_best_forecaster': None,
        'overall_best_anomaly_detector': None,
        'key_insights': [],
        'recommendations_for_improvement': []
    }

    if not forecast_comparison_df.empty:
        report['forecasting_summary'] = forecast_comparison_df.to_dict()
        # Determine best forecaster based on a primary metric, e.g., MAE (lower is better)
        primary_forecast_metric = 'mae_denorm' if 'mae_denorm' in forecast_comparison_df.columns else 'mae_scaled'
        if primary_forecast_metric in forecast_comparison_df.columns:
            report['overall_best_forecaster'] = forecast_comparison_df[primary_forecast_metric].idxmin()
            report['key_insights'].append(f"Best forecasting model ({primary_forecast_metric}): {report['overall_best_forecaster']}")
        else:
            report['key_insights'].append("Could not determine best forecasting model due to missing MAE metrics.")


    if not anomaly_comparison_df.empty:
        report['anomaly_detection_summary'] = anomaly_comparison_df.to_dict()
        # Determine best anomaly detector based on F1-score (higher is better)
        if 'F1 Score' in anomaly_comparison_df.columns: # Ensure column name matches exactly
            report['overall_best_anomaly_detector'] = anomaly_comparison_df['F1 Score'].idxmax()
            report['key_insights'].append(f"Best anomaly detection model (F1 Score): {report['overall_best_anomaly_detector']}")

            
            best_detector_stats = anomaly_comparison_df.loc[report['overall_best_anomaly_detector']]
            if best_detector_stats['Recall'] < 0.5:
                report['key_insights'].append(f"{report['overall_best_anomaly_detector']} has low recall ({best_detector_stats['Recall']:.3f}), missing many anomalies.")
            if best_detector_stats['Precision'] < 0.5:
                report['key_insights'].append(f"{report['overall_best_anomaly_detector']} has low precision ({best_detector_stats['Precision']:.3f}), many false alarms.")
        else:
            report['key_insights'].append("Could not determine best anomaly detection model due to missing F1 Score metric.")


    report['recommendations_for_improvement'] = [
        "Further tune hyperparameters for all models.",
        "Explore more diverse data augmentation techniques if real-world data is limited.",
        "Investigate feature importance and potentially engineer new relevant features.",
        "If Autoencoder recall remains low, explore different anomaly detection algorithms or ensemble methods."
    ]
    
    return report

def run_evaluation_phase():
    """Main function to run the final evaluation and comparison phase."""
    print("\n" + "="*50)
    print("Starting Final Model Evaluation & Comparison Phase")
    print("="*50 + "\n")
    
    start_time_main = time.time()

    # 1. Load all necessary data and pre-computed results/models
    loaded_eval_data = load_all_data_for_evaluation()
    if 'error' in loaded_eval_data:
        print(f"Critical error during data loading: {loaded_eval_data['error']}. Aborting evaluation.")
        return

    # 2. Consolidate and Compare Forecasting Metrics
    print("\n--- Comparing Forecasting Models ---")
    forecasting_metric_files = {
        'LSTM': RESULTS_DIR / 'lstm_metrics.json',
        'TCN': RESULTS_DIR / 'tcn_metrics.json'
    }
    forecasting_comparison_df = compare_forecasting_metrics_dfs(forecasting_metric_files)
    if not forecasting_comparison_df.empty:
        print("\nForecasting Metrics Comparison:")
        print(forecasting_comparison_df)
    else:
        print("No forecasting metrics to compare.")

    # 3. Consolidate and Compare Anomaly Detection Metrics
    print("\n--- Comparing Anomaly Detection Models ---")
    
    anomaly_metric_files = {
        'Autoencoder': RESULTS_DIR / 'autoencoder_anomaly_model_metrics_ad.json' 
    }
    
    all_anomaly_metrics_path = RESULTS_DIR / 'all_anomaly_metrics.csv'
    if all_anomaly_metrics_path.exists():
        anomaly_comparison_df = pd.read_csv(all_anomaly_metrics_path).set_index('Model')
        print("\nAnomaly Detection Metrics Comparison (from all_anomaly_metrics.csv):")
        print(anomaly_comparison_df)
        
        if not anomaly_comparison_df.empty:
            plot_cols_ad = ['Precision', 'Recall', 'F1 Score', 'AUC Score', 'Average Precision']
            
            plot_cols_ad_present = [col for col in plot_cols_ad if col in anomaly_comparison_df.columns]
            if plot_cols_ad_present:
                anomaly_comparison_df[plot_cols_ad_present].plot(kind='bar', figsize=(12, 7))
                plt.title('Anomaly Detection Models Comparison (Final)')
                plt.ylabel('Metric Value')
                plt.xticks(rotation=45, ha='right')
                plt.ylim(0, 1.05) 
                plt.tight_layout()
                plt.savefig(RESULTS_DIR / 'final_anomaly_detection_models_comparison.png')
                plt.close()

    else:
        print(f"Warning: {all_anomaly_metrics_path} not found. Cannot generate full anomaly detection comparison.")
        anomaly_comparison_df = pd.DataFrame()


    
    print("\n--- Generating Final Summary Report ---")
    final_report_data = generate_final_summary_report(forecasting_comparison_df, anomaly_comparison_df)
    
    
    with open(RESULTS_DIR / 'final_evaluation_summary_report.json', 'w') as f:
        json.dump(final_report_data, f, indent=4, default=str) 
    print(f"Final summary report saved to {RESULTS_DIR / 'final_evaluation_summary_report.json'}")

    
    print("\nKey Insights from Final Report:")
    for insight in final_report_data.get('key_insights', []):
        print(f"- {insight}")
    
    print("\nRecommendations for Improvement:")
    for rec in final_report_data.get('recommendations_for_improvement', []):
        print(f"- {rec}")

    if PRINT_PARAMS['show_timing']:
        print(f"\nTotal evaluation phase execution time: {time.time() - start_time_main:.2f} seconds")
    
    print("\n" + "="*50)
    print("Final Model Evaluation & Comparison Phase Completed!")
    print("="*50)


run_evaluation_phase()